<a href="https://colab.research.google.com/github/AlexKitipov/finrl-test/blob/main/Copilot_x_DTASC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 📦 Инсталиране на нужните библиотеки
!pip install -q stable-baselines3 sb3-contrib pandas numpy matplotlib yfinance psutil gym

# 🔗 Монтиране на Google Drive
from google.colab import drive

# Attempt to unmount first in case it's already mounted or the previous attempt failed
!fusermount -u /content/drive
drive.mount('/content/drive', force_remount=True)

# 📚 Импортиране на модули
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import psutil
from datetime import datetime

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
import gym

# 📁 Основни пътища
base_path = "/content/drive/MyDrive/DTASC"
os.makedirs(base_path, exist_ok=True)

# 📂 Проверка за налични файлове
print("📂 Съдържание на папката:")
!ls -lh "$base_path"

# 🕒 Старт на сесията
print(f"🚀 Сесията започна: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.8/92.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

ValueError: mount failed

In [ ]:
!fusermount -u /content/drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


In [ ]:
!ls "$base_path"


In [ ]:
!find "$base_path" -name "X_test.csv"


In [ ]:
!ls -lh "/content/drive/MyDrive/DTASC"


In [ ]:
import os

# Създаване на нужните директории, ако не съществуват
os.makedirs("data", exist_ok=True)
os.makedirs("x_test", exist_ok=True)
os.makedirs("models", exist_ok=True)
os.makedirs("results", exist_ok=True)


In [ ]:
!pip install cartopy
import cartopy

In [ ]:
# https://pypi.python.org/pypi/pydot
!apt-get -qq install -y graphviz && pip install pydot
import pydot

In [ ]:
# https://pypi.python.org/pypi/libarchive
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
import libarchive

In [ ]:
!apt-get -qq install -y libfluidsynth1

In [ ]:
!pip install matplotlib-venn

In [ ]:
df = pd.read_csv(f"{base_path}/X_test.csv", sep=";")
print("📋 Колони в X_test.csv:", df.columns.tolist())


In [ ]:
import threading
import yfinance as yf
import pandas as pd
import numpy as np
import time

def stream_to_dtasc(ticker="EURUSD=X", interval="40m", period="1d", refresh_sec=120, max_cycles=5):
    """
    Стартира фонов поток от YFinance, който периодично обновява 'X_test.csv'
    с 12 входни колони за DTASC.
    """
    def run():
        print("🔄 Стартира фонов поток към DTASC...")
        for i in range(max_cycles):
            try:
                df = yf.download(ticker, interval=interval, period=period, progress=False)
                df = df[["Open", "High", "Low", "Close", "Volume"]].dropna()

                # Технически индикатори
                delta = df["Close"].diff()
                gain = delta.where(delta > 0, 0)
                loss = -delta.where(delta < 0, 0)
                avg_gain = gain.rolling(window=14).mean()
                avg_loss = loss.rolling(window=14).mean()
                rs = avg_gain / avg_loss
                df["RSI"] = 100 - (100 / (1 + rs))

                ema12 = df["Close"].ewm(span=12, adjust=False).mean()
                ema26 = df["Close"].ewm(span=26, adjust=False).mean()
                df["MACD"] = ema12 - ema26

                rolling_mean = df["Close"].rolling(window=20).mean()
                rolling_std = df["Close"].rolling(window=20).std()
                df["Bollinger_Middle"] = rolling_mean
                df["Bollinger_Upper"] = rolling_mean + (2 * rolling_std)
                df["Bollinger_Lower"] = rolling_mean - (2 * rolling_std)

                df["Balance"] = 100000.0  # фиктивна стойност

                df.dropna(inplace=True)

                # Записваме само нужните колони
                features = ["Open", "High", "Low", "Close", "Volume", "RSI", "MACD",
                            "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower", "Balance"]
                df[features].to_csv("X_test.csv", sep=";", index=False)

                print(f"✅ Обновен 'X_test.csv' ({df.shape[0]} реда) | Цикъл {i+1}/{max_cycles}")
            except Exception as e:
                print(f"❌ Грешка при извличане: {e}")
            time.sleep(refresh_sec)
        print("🏁 Потокът към DTASC приключи.")

    t = threading.Thread(target=run)
    t.start()


In [ ]:
stream_to_dtasc(
    ticker="EURUSD=X",
    interval="40m",
    period="1d",
    refresh_sec=120,
    max_cycles=10
)


In [ ]:
stream_to_dtasc(ticker="EURUSD=X", interval="40m", period="1d", refresh_sec=120, max_cycles=5)


In [ ]:
# 📚 Импортираме нужните библиотеки
import yfinance as yf
import pandas as pd
import numpy as np
import os

# 🔧 Конфигурация
symbol = "EURUSD=X"
start_date = "2022-01-01"
end_date = "2024-12-31"
interval = "1d"  # Дневни данни

# ⏬ 1. Изтегляне на данни от Yahoo Finance
df = yf.download(symbol, start=start_date, end=end_date, interval=interval)

# 📦 2. Почистване и подготовка
df.dropna(inplace=True)
df.reset_index(inplace=True)
df = df.rename(columns={"Date": "Time"})
df = df[["Time", "Open", "High", "Low", "Close", "Volume"]]

# 🧠 3. Добавяне на технически индикатори
delta = df["Close"].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
df["RSI"] = 100 - (100 / (1 + rs))

ema12 = df["Close"].ewm(span=12, adjust=False).mean()
ema26 = df["Close"].ewm(span=26, adjust=False).mean()
df["MACD"] = ema12 - ema26

rolling_mean = df["Close"].rolling(window=20).mean()
rolling_std = df["Close"].rolling(window=20).std()
df["Bollinger_Middle"] = rolling_mean
df["Bollinger_Upper"] = rolling_mean + (2 * rolling_std)
df["Bollinger_Lower"] = rolling_mean - (2 * rolling_std)

# 🧮 4. Добавяне на фиктивна колона Balance
df["Balance"] = 100000.0  # Може да стане динамична по време на симулация

# 🧹 5. Премахваме редовете с NaN (заради индикаторите)
df.dropna(inplace=True)

# 💾 6. Записваме пълните данни
df.to_csv("/content/eurusd_data.csv", sep=";", index=False)
print(f"✅ Записани {len(df)} реда в eurusd_data.csv")

# 📤 7. Извличаме последния ред като сигнал
features = ["Open", "High", "Low", "Close", "Volume", "RSI", "MACD",
            "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower", "Balance"]
signal_row = df[features].iloc[[-1]]
signal_row.to_csv("dtasc_signals.csv", sep=";", index=False)
print("✅ Създаден сигнален файл: dtasc_signals.csv")

# 📊 Преглед на сигнала
print("\n📊 Сигнал за inference:\n")
print(signal_row)


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import threading
import time

def stream_to_dtasc(ticker="EURUSD=X", interval="40m", period="1d", refresh_sec=120, max_cycles=1):
    def run():
        for i in range(max_cycles):
            try:
                df = yf.download(ticker, interval=interval, period=period)
                df.dropna(inplace=True)

                # Технически индикатори
                df["RSI"] = df["Close"].rolling(window=14).apply(lambda x: 100 - (100 / (1 + (x.diff().clip(lower=0).sum() / abs(x.diff().clip(upper=0)).sum()))), raw=False)
                df["MACD"] = df["Close"].ewm(span=12).mean() - df["Close"].ewm(span=26).mean()
                df["Bollinger_Middle"] = df["Close"].rolling(window=20).mean()
                df["Bollinger_Upper"] = df["Bollinger_Middle"] + 2 * df["Close"].rolling(window=20).std()
                df["Bollinger_Lower"] = df["Bollinger_Middle"] - 2 * df["Close"].rolling(window=20).std()
                df["Price_Change"] = df["Close"].pct_change()
                df["Balance"] = 100000.0

                df.dropna(inplace=True)

                # 📦 Избор на пълните 12 входа
                features = ["Open", "High", "Low", "Close", "Volume", "RSI", "MACD",
                            "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower", "Balance", "Price_Change"]

                df[features].to_csv("/content/drive/MyDrive/DTASC/X_test.csv", sep=";", index=False)
                print(f"✅ X_test.csv обновен ({len(df)} реда)")

            except Exception as e:
                print(f"⚠️ Грешка при изтегляне: {e}")

            time.sleep(refresh_sec)

    t = threading.Thread(target=run)
    t.start()


In [ ]:
features = ["Open", "High", "Low", "Close", "Volume", "RSI", "MACD",
            "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower", "Balance", "Price_Change"]


In [ ]:
!pip install "shimmy>=2.0"


In [ ]:
import gym
from gym import spaces
import numpy as np
import pandas as pd
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

# 1️⃣ Зареждане и обработка на данните
df = pd.read_csv("eurusd_data.csv", sep=";")
df["Time"] = pd.to_datetime(df["Time"])
df.set_index("Time", inplace=True)

# Преобразуване на числовите колони
for col in ["Open", "High", "Low", "Close", "Volume", "Balance"]:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# RSI
delta = df["Close"].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
df["RSI"] = 100 - (100 / (1 + rs))

# MACD
ema12 = df["Close"].ewm(span=12, adjust=False).mean()
ema26 = df["Close"].ewm(span=26, adjust=False).mean()
df["MACD"] = ema12 - ema26

# Bollinger Bands (всички 3 линии)
rolling_mean = df["Close"].rolling(window=20).mean()
rolling_std = df["Close"].rolling(window=20).std()
df["Bollinger_Middle"] = rolling_mean
df["Bollinger_Upper"] = rolling_mean + (2 * rolling_std)
df["Bollinger_Lower"] = rolling_mean - (2 * rolling_std)

# Price Change
df["Price_Change"] = df["Close"].diff()

df.dropna(inplace=True)

# 2️⃣ Подготовка на входа
features = [
    "Open", "High", "Low", "Close", "Volume", "Balance",
    "RSI", "MACD", "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower", "Price_Change"
]
X = df[features].to_numpy().astype(np.float32)

# 3️⃣ Създаване на RL среда
class TradingEnv(gym.Env):
    def __init__(self, data):
        super(TradingEnv, self).__init__()
        self.data = data
        self.current_step = 0
        self.action_space = spaces.Discrete(2)  # 0 = SELL, 1 = BUY
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(data.shape[1],), dtype=np.float32)

    def reset(self):
        self.current_step = 0
        return self.data[self.current_step]

    def step(self, action):
        self.current_step += 1
        done = self.current_step >= len(self.data) - 1
        reward = np.random.randn()  # Можеш да замениш с по-смислена логика
        obs = self.data[self.current_step] if not done else np.zeros_like(self.data[0])
        return obs, reward, done, {}

    def render(self, mode="human"):
        pass

# 4️⃣ Обучение на модела
env = DummyVecEnv([lambda: TradingEnv(X)])
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)

# 5️⃣ Запазване
model.save("ppo_model_12_features_bollinger")
print("✅ Моделът е обучен и запазен като 'ppo_model_12_features_bollinger.zip'")


In [ ]:
class TradingEnv(gym.Env):
    def __init__(self, data, initial_balance=100000.0, commission=0.001):
        super(TradingEnv, self).__init__()
        self.data = data
        self.initial_balance = initial_balance
        self.commission = commission
        self.action_space = spaces.Discrete(2)  # 0 = SELL, 1 = BUY
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(data.shape[1],), dtype=np.float32)
        self.reset()

    def reset(self):
        self.current_step = 0
        self.balance = self.initial_balance
        self.position = 0  # 0 = no position, 1 = long
        self.entry_price = 0.0
        return self.data[self.current_step]

    def step(self, action):
        self.current_step += 1
        done = self.current_step >= len(self.data) - 1
        current_price = self.data[self.current_step][3]  # Close price

        reward = 0.0

        if action == 1 and self.position == 0:
            # Open long position
            self.position = 1
            self.entry_price = current_price
            self.balance -= current_price * self.commission
        elif action == 0 and self.position == 1:
            # Close long position
            profit = current_price - self.entry_price
            reward = profit - current_price * self.commission
            self.balance += profit
            self.position = 0
            self.entry_price = 0.0

        obs = self.data[self.current_step] if not done else np.zeros_like(self.data[0])
        return obs, reward, done, {"balance": self.balance}


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from stable_baselines3 import PPO

# Зареждане на данни и модел
# Load data from eurusd_data.csv which contains the necessary columns
df = pd.read_csv("/content/eurusd_data.csv", sep=";")

# Convert Time column to datetime and set as index
df["Time"] = pd.to_datetime(df["Time"])
df.set_index("Time", inplace=True)

# Convert relevant columns to numeric, coercing errors to NaN
for col in ["Open", "High", "Low", "Close", "Volume", "Balance"]:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Calculate technical indicators and Price Change (as done during training)
# RSI
delta = df["Close"].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
# Handle division by zero for rs
rs = avg_gain / avg_loss.replace(0, np.nan)
df["RSI"] = 100 - (100 / (1 + rs))

# MACD
ema12 = df["Close"].ewm(span=12, adjust=False).mean()
ema26 = df["Close"].ewm(span=26, adjust=False).mean()
df["MACD"] = ema12 - ema26

# Bollinger Bands
rolling_mean = df["Close"].rolling(window=20).mean()
rolling_std = df["Close"].rolling(window=20).std()
df["Bollinger_Middle"] = rolling_mean
df["Bollinger_Upper"] = rolling_mean + (2 * rolling_std)
df["Bollinger_Lower"] = rolling_mean - (2 * rolling_std)

# Price Change
df["Price_Change"] = df["Close"].diff()

# Define the list of 12 features expected by the model
features = [
    "Open", "High", "Low", "Close", "Volume", "Balance",
    "RSI", "MACD", "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower", "Price_Change"
]

# Drop any rows that might have resulted in NaN after calculating indicators
df.dropna(subset=features, inplace=True)

# Ensure all 12 features are present and in the correct order
df = df[features].copy()

# Подготовка на входа
X = df.to_numpy().astype(np.float32)

# Load the correct model file that was trained with 12 features
model = PPO.load("ppo_model_12_features_bollinger.zip")

# Генериране на действия
actions = []
# Ensure the input observation has the correct shape (1, 12) for prediction
for obs in X:
    action, _ = model.predict(obs.reshape(1, -1), deterministic=True)
    actions.append(action)

# Добавяне на действията към DataFrame
# Reset index of df before adding the 'Action' column to ensure alignment
df.reset_index(drop=False, inplace=True) # Keep the 'Time' column
df["Action"] = actions

# 🎨 Визуализация
plt.figure(figsize=(14, 6))
plt.plot(df.index, df["Close"].values, label="Close Price", color="black")

# BUY точки
buy_signals = df[df["Action"] == 1]
plt.scatter(buy_signals.index, buy_signals["Close"], marker="^", color="green", label="BUY", alpha=0.8)

# SELL точки
sell_signals = df[df["Action"] == 0]
plt.scatter(sell_signals.index, sell_signals["Close"], marker="v", color="red", label="SELL", alpha=0.8)

plt.title("Действия на модела върху цената на EUR/USD")
plt.xlabel("Индекс")
plt.ylabel("Цена")
plt.legend()
plt.grid()
plt.show()

In [ ]:
def step(self, action):
    self.current_step += 1
    done = self.current_step >= len(self.data) - 1
    price = self.data[self.current_step][3]  # Close

    reward = 0.0
    if action == 1 and self.position == 0:
        self.position = 1
        self.entry_price = price
        self.holding_steps = 0
        self.balance -= price * self.commission
    elif action == 0 and self.position == 1:
        profit = price - self.entry_price
        reward = profit - price * self.commission
        self.balance += profit
        self.position = 0
        self.entry_price = 0.0
    elif self.position == 1:
        self.holding_steps += 1
        unrealized = price - self.entry_price
        if unrealized <= -self.stop_loss or unrealized >= self.take_profit:
            reward = unrealized - price * self.commission
            self.balance += unrealized
            self.position = 0
            self.entry_price = 0.0
        else:
            reward -= self.holding_penalty

    obs = self.data[self.current_step] if not done else np.zeros_like(self.data[0])
    return obs, reward, done, {}


In [ ]:
import pandas as pd

# Load the data which includes the 'Time' column
df_split = pd.read_csv("/content/eurusd_data.csv", sep=";")

# Convert the 'Time' column to datetime objects
df_split["Time"] = pd.to_datetime(df_split["Time"])

# Split the data based on the 'Time' column
train_df = df_split[df_split["Time"] < "2024-01-01"]
test_df = df_split[df_split["Time"] >= "2024-01-01"]

print(f"Training data shape: {train_df.shape}")
print(f"Testing data shape: {test_df.shape}")

In [ ]:
model = PPO("MlpPolicy", env, learning_rate=0.0001, n_steps=4096, gamma=0.99, verbose=1)


In [ ]:
# Зареждане на пълните данни
df = pd.read_csv("/content/eurusd_data.csv", sep=";", header=1) # Skip the first row (original header)
df.columns = ["Time", "Open", "High", "Low", "Close", "Volume", "RSI", "MACD", "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower", "Balance"] # Assign correct column names
df["Time"] = pd.to_datetime(df["Time"])

# Convert relevant columns to numeric, coercing errors to NaN
for col in ["Open", "High", "Low", "Close", "Volume", "Balance", "RSI", "MACD", "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower"]:
     df[col] = pd.to_numeric(df[col], errors='coerce')

df.set_index("Time", inplace=True)

# Calculate technical indicators (re-calculating here to ensure consistency after loading)
# RSI
delta = df["Close"].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
# Handle division by zero for rs
rs = avg_gain / avg_loss.replace(0, np.nan)
df["RSI"] = 100 - (100 / (1 + rs))

# MACD
ema12 = df["Close"].ewm(span=12, adjust=False).mean()
ema26 = df["Close"].ewm(span=26, adjust=False).mean()
df["MACD"] = ema12 - ema26

# Bollinger Bands
rolling_mean = df["Close"].rolling(window=20).mean()
rolling_std = df["Close"].rolling(window=20).std()
df["Bollinger_Middle"] = rolling_mean
df["Bollinger_Upper"] = rolling_mean + (2 * rolling_std)
df["Bollinger_Lower"] = rolling_mean - (2 * rolling_std)

# Price Change
df["Price_Change"] = df["Close"].diff()


# Разделяне на train/test по дата
train_df = df[df.index < "2024-01-01"].copy()
test_df = df[df.index >= "2024-01-01"].copy()

# Избор на входни характеристики
features = ["Open", "High", "Low", "Close", "Volume", "Balance",
            "RSI", "MACD", "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower", "Price_Change"]

# Drop rows with NaN values that resulted from calculating indicators
train_df.dropna(subset=features, inplace=True)
test_df.dropna(subset=features, inplace=True)


X_train = train_df[features].to_numpy().astype(np.float32)
X_test = test_df[features].to_numpy().astype(np.float32)

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
import gym
from gym import spaces

# TradingEnv с фиксирана логика
class TradingEnv(gym.Env):
    def __init__(self, data):
        super().__init__()
        self.data = data
        self.current_step = 0
        self.action_space = spaces.Discrete(2)  # 0 = SELL, 1 = BUY
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(data.shape[1],), dtype=np.float32)

    def reset(self):
        self.current_step = 0
        return self.data[self.current_step]

    def step(self, action):
        self.current_step += 1
        done = self.current_step >= len(self.data) - 1
        reward = np.random.randn()  # Можеш да замениш с по-смислена логика
        obs = self.data[self.current_step] if not done else np.zeros_like(self.data[0])
        return obs, reward, done, {}

    def render(self, mode="human"):
        pass

# Обучение
env_train = DummyVecEnv([lambda: TradingEnv(X_train)])
model = PPO("MlpPolicy", env_train, verbose=1)
model.learn(total_timesteps=10000)
model.save("/content/drive/MyDrive/DTASC/ppo_model_trained_on_train_df")


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
import gym
from gym import spaces
import os
import yfinance as yf # Import yfinance for data generation

# --- Data Generation (Copied from cell_id: 7910ff20) ---
print("Generating data...")
symbol = "EURUSD=X"
start_date = "2022-01-01"
end_date = "2024-12-31"
interval = "1d"  # Дневни данни

try:
    df_new = yf.download(symbol, start=start_date, end=end_date, interval=interval)

    # 📦 2. Почистване и подготовка
    df_new.dropna(inplace=True)
    df_new.reset_index(inplace=True)
    df_new = df_new.rename(columns={"Date": "Time"})
    df_new = df_new[["Time", "Open", "High", "Low", "Close", "Volume"]]

    # 🧠 3. Добавяне на технически индикатори
    delta = df_new["Close"].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss.replace(0, np.nan) # Handle division by zero
    df_new["RSI"] = 100 - (100 / (1 + rs))

    ema12 = df_new["Close"].ewm(span=12, adjust=False).mean()
    ema26 = df_new["Close"].ewm(span=26, adjust=False).mean()
    df_new["MACD"] = ema12 - ema26

    rolling_mean = df_new["Close"].rolling(window=20).mean()
    rolling_std = df_new["Close"].rolling(window=20).std()
    df_new["Bollinger_Middle"] = rolling_mean
    df_new["Bollinger_Upper"] = rolling_mean + (2 * rolling_std)
    df_new["Bollinger_Lower"] = rolling_mean - (2 * rolling_std)

    # Price Change - Ensure this is calculated
    df_new["Price_Change"] = df_new["Close"].diff()

    # 🧮 4. Добавяне на фиктивна колона Balance
    df_new["Balance"] = 100000.0  # Може да стане динамична по време на симулация

    # 🧹 5. Премахваме редовете с NaN (заради индикаторите)
    df_new.dropna(inplace=True)

    # Ensure the order of columns matches the features list needed for the next step
    features_for_save = [
        "Time", "Open", "High", "Low", "Close", "Volume", "Balance",
        "RSI", "MACD", "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower", "Price_Change"
    ]
    df_new = df_new[features_for_save].copy()

    # 💾 6. Записваме пълните данни в /content/
    data_path = "/content/eurusd_data_new.csv" # Save to /content/
    df_new.to_csv(data_path, sep=";", index=False)
    print(f"✅ Записани {len(df_new)} реда в {data_path}")

except Exception as e:
    print(f"❌ Error during data generation: {e}")
    # If data generation fails, stop execution
    raise


# --- TradingEnv Definition (Copied from successful execution cell h2OuY7yk2bQ5) ---
class TradingEnv(gym.Env):
    def __init__(self, data, initial_balance=100000.0, commission=0.001, stop_loss=0.01, take_profit=0.02, holding_penalty=0.0001):
        super(TradingEnv, self).__init__()
        self.data = data
        self.initial_balance = initial_balance
        self.commission = commission
        self.stop_loss = stop_loss
        self.take_profit = take_profit
        self.holding_penalty = holding_penalty

        self.action_space = spaces.Discrete(2)  # 0 = SELL, 1 = BUY
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(data.shape[1],), dtype=np.float32)

        self.reset()

    def reset(self):
        self.current_step = 0
        self.balance = self.initial_balance
        self.position = 0  # 0 = no position, 1 = long
        self.entry_price = 0.0
        self.holding_steps = 0
        return self.data[self.current_step]

    def step(self, action):
        self.current_step += 1
        done = self.current_step >= len(self.data) - 1
        obs = self.data[self.current_step] if not done else np.zeros_like(self.data[0])
        price = self.data[self.current_step][3]  # Close

        reward = 0.0

        if action == 1 and self.position == 0:
            # BUY
            self.position = 1
            self.entry_price = price
            self.holding_steps = 0
            self.balance -= price * self.commission

        elif action == 0 and self.position == 1:
            # SELL
            profit = price - self.entry_price
            reward = profit - price * self.commission
            self.balance += profit
            self.position = 0
            self.entry_price = 0.0

        elif self.position == 1:
            # HOLD
            self.holding_steps += 1
            unrealized = price - self.entry_price

            # Stop-loss
            if unrealized <= -self.stop_loss * self.entry_price:
                reward = unrealized - price * self.commission
                self.balance += unrealized
                self.position = 0
                self.entry_price = 0.0

            # Take-profit
            elif unrealized >= self.take_profit * self.entry_price:
                reward = unrealized - price * self.commission
                self.balance += unrealized
                self.position = 0
                self.entry_price = 0.0

            else:
                reward -= self.holding_penalty * self.holding_steps

        return obs, reward, done, {}

    def render(self, mode="human"):
        print(f"Step: {self.current_step}, Balance: {self.balance:.2f}, Position: {self.position}, Entry: {self.entry_price:.5f}")


# --- Data Loading and Preparation for Training/Testing ---
# Define the path to the data file generated above
data_path_load = "/content/eurusd_data_new.csv" # Load from /content/


# Load the data from the correct path
try:
    df = pd.read_csv(data_path_load, sep=";")

    # Check if the DataFrame has the expected columns, considering the potential multi-level header from yfinance
    # Based on the data generation code above, the header should be single level now, but keep the check for robustness
    if df.columns[0].startswith('Price'):
        print("Detected potential multi-level header during loading, reloading with header=1.")
        df = pd.read_csv(data_path_load, sep=";", header=1)
        # Assign correct column names
        df.columns = ["Time", "Open", "High", "Low", "Close", "Volume", "Balance", "RSI", "MACD", "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower", "Price_Change"]
    elif 'Time' not in df.columns:
         print("Time column not found during loading, assuming first column is Time.")
         df.columns = ["Time"] + df.columns[1:].tolist()


    # Convert the 'Time' column to datetime objects and set as index
    df["Time"] = pd.to_datetime(df["Time"])
    df.set_index("Time", inplace=True)

    # Ensure relevant columns are numeric, coercing errors to NaN
    for col in ["Open", "High", "Low", "Close", "Volume", "Balance", "RSI", "MACD", "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower", "Price_Change"]:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Define the list of 12 features expected by the TradingEnv
    features = [
        "Open", "High", "Low", "Close", "Volume", "Balance",
        "RSI", "MACD", "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower", "Price_Change"
    ]

    # Split the DataFrame into training and testing sets based on time
    train_df = df[df.index < "2024-01-01"].copy()
    test_df = df[df.index >= "2024-01-01"].copy()

    # Drop rows with NaN values from both sets based on features
    train_df.dropna(subset=features, inplace=True)
    test_df.dropna(subset=features, inplace=True)

    # Convert DataFrames to NumPy arrays
    X_train = train_df[features].to_numpy().astype(np.float32)
    X_test = test_df[features].to_numpy().astype(np.float32)

    print(f"X_train shape: {X_train.shape}")
    print(f"X_test shape: {X_test.shape}")

except FileNotFoundError:
    # This should ideally not happen if data generation was successful
    print(f"Error: The data file {data_path_load} was not found during loading.")
    raise # Re-raise the exception to stop execution
except Exception as e:
    print(f"An error occurred during data loading and processing: {e}")
    raise # Re-raise the exception to stop execution


# --- Model Training ---
print("\nTraining the model...")
env_train = DummyVecEnv([lambda: TradingEnv(X_train)])
model = PPO("MlpPolicy", env_train, verbose=1)
model.learn(total_timesteps=10000) # You can adjust total_timesteps as needed

# Save the trained model
model_save_path = "/content/drive/MyDrive/DTASC/ppo_model_trained_on_train_df.zip"
model.save(model_save_path)
print(f"✅ Моделът е обучен и запазен като '{model_save_path}'")


# --- Model Evaluation ---
print("\nEvaluating the model on test data...")
# Load the trained model (now it should exist)
model = PPO.load(model_save_path)

# Simulation on test data
env_test = TradingEnv(X_test)
obs = env_test.reset()
rewards = []
balance = env_test.initial_balance # Use the initial balance from the environment
equity = [balance] # Start equity with initial balance
position = 0 # Track position for equity calculation
entry_price = 0.0 # Track entry price for equity calculation

for i in range(len(X_test) - 1):
    action, _ = model.predict(obs.reshape(1, -1), deterministic=True)
    action = int(action[0]) if isinstance(action, np.ndarray) else int(action) # Ensure action is integer

    obs, reward, done, info = env_test.step(action) # Get info dictionary

    rewards.append(reward)

    # Update position and entry price based on action for equity calculation
    current_price = X_test[i+1][3] # Close price of the next step
    if action == 1 and position == 0: # BUY
        position = 1
        entry_price = current_price # Entry price is the price of the step *after* the BUY action
    elif action == 0 and position == 1: # SELL
        position = 0
        entry_price = 0.0 # Reset entry price on SELL

    # Calculate current equity: balance + profit/loss from current position
    current_equity = env_test.balance + (X_test[i+1][3] - env_test.entry_price if env_test.position else 0)
    equity.append(current_equity)


    if done:
        break

# Visualization
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(np.cumsum(rewards), label="Кумулативна награда")
plt.title("📊 Оценка на модела върху тестови данни (Кумулативна награда)")
plt.xlabel("Стъпка")
plt.ylabel("Награда")
plt.grid()
plt.legend()

plt.subplot(1, 2, 2)
# Plot equity against the length of equity list
plt.plot(equity, label="Баланс (Equity)")
plt.title("💰 Оценка на модела върху тестови данни (Баланс)")
plt.xlabel("Стъпка")
plt.ylabel("Баланс")
plt.grid()
plt.legend()

plt.tight_layout()
plt.show()

# Calculate and print trading metrics (Example - you can add more)
total_reward = np.sum(rewards)
final_balance = equity[-1] if equity else env_test.initial_balance
initial_balance = env_test.initial_balance
profit_loss = final_balance - initial_balance

print("\n--- Търговски метрики върху тестови данни ---")
print(f"Начален баланс: {initial_balance:.2f}")
print(f"Краен баланс: {final_balance:.2f}")
print(f"Обща печалба/загуба: {profit_loss:.2f}")
print(f"Обща кумулативна награда: {total_reward:.2f}")
print("--------------------------------------------")

In [ ]:
import pandas as pd
import numpy as np
from stable_baselines3 import PPO
import matplotlib.pyplot as plt

# 📁 Път до данните и модела
base_path = "/content/drive/MyDrive/DTASC"

# 📊 Зареждане на последни данни
df = pd.read_csv(f"{base_path}/X_test.csv", sep=";")

# Convert relevant columns to numeric, coercing errors to NaN
for col in ["Open", "High", "Low", "Close", "Volume", "Balance"]:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Calculate technical indicators (as done during training)
# RSI
delta = df["Close"].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
# Handle division by zero for rs
rs = avg_gain / avg_loss.replace(0, np.nan)
df["RSI"] = 100 - (100 / (1 + rs))

# MACD
ema12 = df["Close"].ewm(span=12, adjust=False).mean()
ema26 = df["Close"].ewm(span=26, adjust=False).mean()
df["MACD"] = ema12 - ema26

# Bollinger Bands
rolling_mean = df["Close"].rolling(window=20).mean()
rolling_std = df["Close"].rolling(window=20).std()
df["Bollinger_Middle"] = rolling_mean
df["Bollinger_Upper"] = rolling_mean + (2 * rolling_std)
df["Bollinger_Lower"] = rolling_mean - (2 * rolling_std)

# Price Change
df["Price_Change"] = df["Close"].diff()


features = ["Open", "High", "Low", "Close", "Volume", "RSI", "MACD",
            "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower", "Balance", "Price_Change"]

# Drop any rows that might have resulted in NaN after calculating indicators
df.dropna(subset=features, inplace=True)

X = df[features].to_numpy().astype(np.float32)

# 🤖 Зареждане на модела
# Assuming ppo_model_12_features_bollinger.zip is the correct model with 12 features
model = PPO.load("ppo_model_12_features_bollinger.zip")


# 💰 Симулация на действия и награди
balance = 100000.0
position = 0
entry_price = 0.0
commission = 0.001
rewards = []
equity = []
actions = []

for i in range(len(X)): # Iterate through all data points after dropping NaNs
    obs = X[i].reshape(1, -1)
    action, _ = model.predict(obs, deterministic=True)
    action = int(action)
    price = X[i][3]  # Close price is the 4th feature (index 3)

    reward = 0.0
    # Action 1: BUY, Action 0: SELL
    if action == 1 and position == 0:  # BUY
        position = 1
        entry_price = price
        balance -= price * commission
    elif action == 0 and position == 1:  # SELL
        profit = price - entry_price
        reward = profit - price * commission
        balance += profit
        position = 0
        entry_price = 0.0
    # HOLD (if action is not BUY when position is 0, or not SELL when position is 1)

    rewards.append(reward)
    # Calculate current equity: balance + profit from current position
    current_equity = balance + (price - entry_price if position else 0)
    equity.append(current_equity)
    actions.append(action)


# 📈 Визуализация
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
# Plotting rewards, starting from the second data point as the first action is based on the first observation
plt.plot(np.cumsum(rewards), label="Кумулативна награда")
plt.title("📈 Кумулативна награда")
plt.xlabel("Стъпка")
plt.ylabel("Награда")
plt.grid()
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(equity, label="Баланс")
plt.title("💰 Баланс във времето")
plt.xlabel("Стъпка")
plt.ylabel("Баланс")
plt.grid()
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
if np.sum(rewards) < -100:
    print("⚠️ Дообучаване на модела...")
    env = DummyVecEnv([lambda: TradingEnv(X)])
    model.set_env(env)
    model.learn(total_timesteps=2000)
    model.save(f"{base_path}/ppo_real_model.zip")


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/DTASC/X_test.csv", sep=";")
print("📊 Колони:", df.columns.tolist())
print("🧪 Последен ред:\n", df.tail(1))


In [ ]:
import gym
from gym import spaces
import numpy as np

class TradingEnv(gym.Env):
    def __init__(self, data, initial_balance=100000.0, commission=0.001, stop_loss=0.01, take_profit=0.02, holding_penalty=0.0001):
        super(TradingEnv, self).__init__()
        self.data = data
        self.initial_balance = initial_balance
        self.commission = commission
        self.stop_loss = stop_loss
        self.take_profit = take_profit
        self.holding_penalty = holding_penalty

        self.action_space = spaces.Discrete(2)  # 0 = SELL, 1 = BUY
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(data.shape[1],), dtype=np.float32)

        self.reset()

    def reset(self):
        self.current_step = 0
        self.balance = self.initial_balance
        self.position = 0  # 0 = no position, 1 = long
        self.entry_price = 0.0
        self.holding_steps = 0
        return self.data[self.current_step]

    def step(self, action):
        self.current_step += 1
        done = self.current_step >= len(self.data) - 1
        obs = self.data[self.current_step] if not done else np.zeros_like(self.data[0])
        price = self.data[self.current_step][3]  # Close

        reward = 0.0

        if action == 1 and self.position == 0:
            # BUY
            self.position = 1
            self.entry_price = price
            self.holding_steps = 0
            self.balance -= price * self.commission

        elif action == 0 and self.position == 1:
            # SELL
            profit = price - self.entry_price
            reward = profit - price * self.commission
            self.balance += profit
            self.position = 0
            self.entry_price = 0.0

        elif self.position == 1:
            # HOLD
            self.holding_steps += 1
            unrealized = price - self.entry_price

            # Stop-loss
            if unrealized <= -self.stop_loss * self.entry_price:
                reward = unrealized - price * self.commission
                self.balance += unrealized
                self.position = 0
                self.entry_price = 0.0

            # Take-profit
            elif unrealized >= self.take_profit * self.entry_price:
                reward = unrealized - price * self.commission
                self.balance += unrealized
                self.position = 0
                self.entry_price = 0.0

            else:
                reward -= self.holding_penalty * self.holding_steps

        return obs, reward, done, {}

    def render(self, mode="human"):
        print(f"Step: {self.current_step}, Balance: {self.balance:.2f}, Position: {self.position}, Entry: {self.entry_price:.5f}")


In [ ]:
import pandas as pd
import numpy as np
from stable_baselines3 import PPO

# Зареждане на тестовите данни
# Load the data which includes the necessary columns
df = pd.read_csv("/content/eurusd_data_new.csv", sep=";") # Load from eurusd_data_new.csv
df["Time"] = pd.to_datetime(df["Time"])
df.set_index("Time", inplace=True)

# Convert relevant columns to numeric, coercing errors to NaN
for col in ["Open", "High", "Low", "Close", "Volume", "Balance"]:
    df[col] = pd.to_numeric(df[col], errors='coerce')


# Calculate technical indicators and Price Change (re-calculating here to ensure consistency after loading)
# RSI
delta = df["Close"].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
# Handle division by zero for rs
rs = avg_gain / avg_loss.replace(0, np.nan)
df["RSI"] = 100 - (100 / (1 + rs))

# MACD
ema12 = df["Close"].ewm(span=12, adjust=False).mean()
ema26 = df["Close"].ewm(span=26, adjust=False).mean()
df["MACD"] = ema12 - ema26

# Bollinger Bands
rolling_mean = df["Close"].rolling(window=20).mean()
rolling_std = df["Close"].rolling(window=20).std()
df["Bollinger_Middle"] = rolling_mean
df["Bollinger_Upper"] = rolling_mean + (2 * rolling_std)
df["Bollinger_Lower"] = rolling_mean - (2 * rolling_std)

# Price Change
df["Price_Change"] = df["Close"].diff()

# Define the list of 12 features expected by the model
features = [
    "Open", "High", "Low", "Close", "Volume", "Balance",
    "RSI", "MACD", "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower", "Price_Change"
]

# Drop any rows that might have resulted in NaN after calculating indicators
df.dropna(subset=features, inplace=True)

# Explicitly select and reorder columns according to the features list
df_processed = df[features].copy()

# Convert all columns to numeric after selection
for col in features:
    df_processed[col] = pd.to_numeric(df_processed[col], errors='coerce')

# Drop rows with NaNs again after numeric conversion
df_processed.dropna(subset=features, inplace=True)


X_test = df_processed.to_numpy().astype(np.float32)

print("Shape of X_test:", X_test.shape)
print("Columns used for X_test:", df_processed.columns.tolist())


# Зареждане на модела
model = PPO.load("/content/drive/MyDrive/DTASC/ppo_model_trained_on_train_df.zip")

# Симулация и логване
log = []
balance = 100000.0
position = 0
entry_price = 0.0
commission = 0.001

for i in range(len(X_test)): # Iterate through all data points
    obs = X_test[i].reshape(1, -1)
    action, _ = model.predict(obs, deterministic=True)
    action = int(action)
    price = X_test[i][3]  # Close price is the 4th feature (index 3)

    reward = 0.0
    note = ""

    # Action 1: BUY, Action 0: SELL
    if action == 1 and position == 0:  # BUY
        position = 1
        entry_price = price
        balance -= price * commission
        note = "BUY"
    elif action == 0 and position == 1:  # SELL
        profit = price - entry_price
        reward = profit - price * commission
        balance += profit
        position = 0
        entry_price = 0.0
        note = "SELL"
    else:
        note = "HOLD"

    # Ensure the index exists before appending to log
    if i < len(df_processed.index):
        log.append({
            "Time": df_processed.index[i],
            "Action": note,
            "Price": price,
            "Balance": balance,
            "Reward": reward,
            "Position": position
        })


# Запис в CSV
log_df = pd.DataFrame(log)
log_df.to_csv("/content/drive/MyDrive/DTASC/dtasc_actions_log.csv", index=False, sep=";")
print("✅ Записан лог файл: dtasc_actions_log.csv")

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import os

# 🔧 Конфигурация - Use the same configuration as the original data generation
symbol = "EURUSD=X"
start_date = "2022-01-01"
end_date = "2024-12-31"
interval = "1d"  # Дневни данни

# ⏬ 1. Изтегляне на данни от Yahoo Finance
df_new = yf.download(symbol, start=start_date, end=end_date, interval=interval)

# 📦 2. Почистване и подготовка
df_new.dropna(inplace=True)
df_new.reset_index(inplace=True)
df_new = df_new.rename(columns={"Date": "Time"})
df_new = df_new[["Time", "Open", "High", "Low", "Close", "Volume"]]

# 🧠 3. Добавяне на технически индикатори
delta = df_new["Close"].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss.replace(0, np.nan) # Handle division by zero
df_new["RSI"] = 100 - (100 / (1 + rs))

ema12 = df_new["Close"].ewm(span=12, adjust=False).mean()
ema26 = df_new["Close"].ewm(span=26, adjust=False).mean()
df_new["MACD"] = ema12 - ema26

rolling_mean = df_new["Close"].rolling(window=20).mean()
rolling_std = df_new["Close"].rolling(window=20).std()
df_new["Bollinger_Middle"] = rolling_mean
df_new["Bollinger_Upper"] = rolling_mean + (2 * rolling_std)
df_new["Bollinger_Lower"] = rolling_mean - (2 * rolling_std)

# Price Change - Ensure this is calculated
df_new["Price_Change"] = df_new["Close"].diff()

# 🧮 4. Добавяне на фиктивна колона Balance
df_new["Balance"] = 100000.0  # Може да стане динамична по време на симулация

# 🧹 5. Премахваме редовете с NaN (заради индикаторите)
df_new.dropna(inplace=True)

# Ensure the order of columns matches the features list
features = [
    "Time", "Open", "High", "Low", "Close", "Volume", "Balance",
    "RSI", "MACD", "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower", "Price_Change"
]
df_new = df_new[features].copy()


# 💾 6. Записваме пълните данни
df_new.to_csv("/content/eurusd_data_new.csv", sep=";", index=False)
print(f"✅ Записани {len(df_new)} реда в eurusd_data_new.csv")

# 📊 Преглед на първите няколко реда и колоните
print("\n📊 Първи 5 реда на новия файл:\n")
print(df_new.head())
print("\n📋 Колони в новия файл:", df_new.columns.tolist())

In [ ]:
!ls -lh "/content/drive/MyDrive/DTASC/"

In [ ]:
import os

model_path = "/content/drive/MyDrive/DTASC/ppo_model_trained_on_train_df.zip"
if os.path.exists(model_path):
    print(f"✅ Model file found at: {model_path}")
else:
    print(f"❌ Model file NOT found at: {model_path}")
    print("Please ensure the file exists and Google Drive is correctly mounted.")

In [ ]:
['Open', 'High', 'Low', 'Close', 'Volume', 'Balance']


In [ ]:
stream_to_dtasc(ticker="EURUSD=X", interval="15m", period="1d", refresh_sec=120, max_cycles=5)


In [ ]:
log_df = pd.read_csv("/content/drive/MyDrive/DTASC/dtasc_actions_log.csv", sep=";")
plt.plot(log_df["Price"], label="Close Price")

buy_signals = log_df[log_df["Action"] == "BUY"]
sell_signals = log_df[log_df["Action"] == "SELL"]

plt.scatter(buy_signals.index, buy_signals["Price"], marker="^", color="green", label="BUY")
plt.scatter(sell_signals.index, sell_signals["Price"], marker="v", color="red", label="SELL")

plt.legend()
plt.title("📈 BUY/SELL сигнали от DTASC")
plt.grid()
plt.show()


In [ ]:
model = PPO.load("/content/drive/MyDrive/DTASC/ppo_real_model.zip")


In [ ]:
from stable_baselines3.common.vec_env import DummyVecEnv
import gym
import pandas as pd
import numpy as np
from stable_baselines3 import PPO # Import PPO

# Зареждане на пълните данни
df_full = pd.read_csv("/content/eurusd_data_new.csv", sep=";") # Load from eurusd_data_new.csv
features = ["Open", "High", "Low", "Close", "Volume", "Balance",
            "RSI", "MACD", "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower", "Price_Change"]

# Ensure all features are present and in the correct order before converting to numpy
df_full = df_full[features].copy()

X = df_full.dropna().to_numpy().astype(np.float32)

# Създаване на среда
class TradingEnv(gym.Env):
    def __init__(self, data):
        super().__init__()
        self.data = data
        self.current_step = 0
        self.action_space = gym.spaces.Discrete(2)
        self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(data.shape[1],), dtype=np.float32)

    def reset(self):
        self.current_step = 0
        return self.data[self.current_step]

    def step(self, action):
        self.current_step += 1
        done = self.current_step >= len(self.data) - 1
        reward = np.random.randn()  # Може да се замени с реална логика
        obs = self.data[self.current_step] if not done else np.zeros_like(self.data[0]) # Handle case when done
        return obs, reward, done, {}

env = DummyVecEnv([lambda: TradingEnv(X)])

# Преобучение
# model needs to be loaded first if not already in scope
model = PPO.load("/content/drive/MyDrive/DTASC/ppo_model_trained_on_train_df.zip") # Load the model
model.set_env(env)
model.learn(total_timesteps=5000)
model.save("/content/drive/MyDrive/DTASC/model_retrain_latest.zip")
print("✅ Моделът е преобучен и запазен.")

In [ ]:
import pandas as pd
import numpy as np
from stable_baselines3 import PPO

# 🔁 Зареждане на X_test.csv
x_test_path = "/content/drive/MyDrive/DTASC/X_test.csv"
df = pd.read_csv(x_test_path, sep=";")

# ✅ Проверка за нужните 12 входа
expected_features = [
    "Open", "High", "Low", "Close", "Volume", "RSI", "MACD",
    "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower", "Balance", "Price_Change"
]

missing = [col for col in expected_features if col not in df.columns]
if missing:
    print(f"⚠️ Липсващи колони в X_test.csv: {missing}")
else:
    # 🧠 Подготовка на входа
    obs = df[expected_features].iloc[[-1]].to_numpy().astype(np.float32)

    # 📦 Зареждане на модела
    model_path = "/content/drive/MyDrive/DTASC/ppo_real_model.zip"
    model = PPO.load(model_path)

    # 🤖 Предсказване на действие
    action, _ = model.predict(obs, deterministic=True)
    action = int(action[0]) if isinstance(action, np.ndarray) else int(action)

    print("📌 Действие от модела:", "BUY 🟢" if action == 1 else "SELL 🔴")


# Task
Подобри предоставения код за обучение на търговски агент с подсилващо обучение, като: 1) дефинираш подобрена среда за търговия (`TradingEnv`) с реалистична логика за наградите (печалба/загуба от затворени позиции, комисионни); 2) разделиш данните от "eurusd_data.csv" на тренировъчен и тестов набор; 3) обучиш нов PPO модел с подобрената среда и тренировъчните данни; 4) оцениш модела върху тестовите данни, като изчислиш търговски метрики; 5) визуализираш резултатите от симулацията върху тестовите данни; 6) (по избор) експериментираш с хиперпараметри или структурата на средата/наградите; 7) (по избор) внедриш логване на действията в CSV файл.

In [ ]:
import gym
from gym import spaces
import numpy as np

class TradingEnv(gym.Env):
    def __init__(self, data, commission=0.0002):
        super(TradingEnv, self).__init__()
        self.data = data
        self.current_step = 0
        self.position = 0  # 0 = flat, 1 = long
        self.entry_price = 0
        self.balance = 100000
        self.commission = commission

        self.action_space = spaces.Discrete(2)  # 0 = SELL, 1 = BUY
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(data.shape[1],), dtype=np.float32)

    def reset(self):
        self.current_step = 0
        self.position = 0
        self.entry_price = 0
        self.balance = 100000
        return self.data[self.current_step]

    def step(self, action):
        price = self.data[self.current_step][3]  # Close
        reward = 0

        if action == 1 and self.position == 0:
            self.position = 1
            self.entry_price = price
            reward = -self.commission
        elif action == 0 and self.position == 1:
            profit = price - self.entry_price
            reward = profit - self.commission
            self.balance += profit
            self.position = 0
            self.entry_price = 0

        self.current_step += 1
        done = self.current_step >= len(self.data) - 1
        obs = self.data[self.current_step] if not done else np.zeros_like(self.data[0])
        return obs, reward, done, {"balance": self.balance}

    def render(self, mode="human"):
        print(f"Step: {self.current_step}, Balance: {self.balance}, Position: {self.position}")

## Дефиниране на подобрена `tradingenv`

### Subtask:
Създайте нова версия на класа `TradingEnv` с реалистична логика за наградите (печалба/загуба от затворени позиции, комисионни).


**Reasoning**:
Define the improved TradingEnv class with realistic reward logic, including commission, stop-loss, take-profit, and holding penalty. This addresses steps 1 through 8 of the subtask.



In [ ]:
import gym
from gym import spaces
import numpy as np

class TradingEnv(gym.Env):
    def __init__(self, data, initial_balance=100000.0, commission=0.001, stop_loss=0.01, take_profit=0.02, holding_penalty=0.0001):
        super(TradingEnv, self).__init__()
        self.data = data
        self.initial_balance = initial_balance
        self.commission = commission
        self.stop_loss = stop_loss
        self.take_profit = take_profit
        self.holding_penalty = holding_penalty

        self.action_space = spaces.Discrete(2)  # 0 = SELL, 1 = BUY
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(data.shape[1],), dtype=np.float32)

        self.reset()

    def reset(self):
        self.current_step = 0
        self.balance = self.initial_balance
        self.position = 0  # 0 = no position, 1 = long
        self.entry_price = 0.0
        self.holding_steps = 0
        return self.data[self.current_step]

    def step(self, action):
        self.current_step += 1
        done = self.current_step >= len(self.data) - 1
        obs = self.data[self.current_step] if not done else np.zeros_like(self.data[0])
        price = self.data[self.current_step][3]  # Close

        reward = 0.0

        if action == 1 and self.position == 0:
            # BUY
            self.position = 1
            self.entry_price = price
            self.holding_steps = 0
            self.balance -= price * self.commission

        elif action == 0 and self.position == 1:
            # SELL
            profit = price - self.entry_price
            reward = profit - price * self.commission
            self.balance += profit
            self.position = 0
            self.entry_price = 0.0

        elif self.position == 1:
            # HOLD
            self.holding_steps += 1
            unrealized = price - self.entry_price

            # Stop-loss
            if unrealized <= -self.stop_loss * self.entry_price:
                reward = unrealized - price * self.commission
                self.balance += unrealized
                self.position = 0
                self.entry_price = 0.0

            # Take-profit
            elif unrealized >= self.take_profit * self.entry_price:
                reward = unrealized - price * self.commission
                self.balance += unrealized
                self.position = 0
                self.entry_price = 0.0

            else:
                reward -= self.holding_penalty * self.holding_steps

        return obs, reward, done, {}

    def render(self, mode="human"):
        print(f"Step: {self.current_step}, Balance: {self.balance:.2f}, Position: {self.position}, Entry: {self.entry_price:.5f}")


In [ ]:
split_index = int(len(X) * 0.8)
X_train = X[:split_index]
X_test = X[split_index:]


## Подготовка на данните за обучение и тестване

### Subtask:
Разделете изтеглените данни (`eurusd_data.csv`) на тренировъчен и тестов набор по времева ос. Уверете се, че всички 12 характеристики (вкл. индикаторите) са правилно изчислени за двата набора.


**Reasoning**:
Load the data, convert time and relevant columns, calculate indicators, define features, split data, drop NaNs, convert to numpy arrays, and print shapes.



In [ ]:
train_env = DummyVecEnv([lambda: TradingEnv(X_train)])
model = PPO("MlpPolicy", train_env, verbose=1)
model.learn(total_timesteps=10000)
model.save("/content/drive/MyDrive/DTASC/ppo_model_trained_on_train_df.zip")


In [ ]:
test_env = TradingEnv(X_test)
obs = test_env.reset()
rewards = []
balances = []
actions = []

for _ in range(len(X_test) - 1):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, info = test_env.step(action)
    rewards.append(reward)
    balances.append(info["balance"])
    actions.append(int(action))
    if done:
        break

import matplotlib.pyplot as plt

plt.figure(figsize=(14, 5))
plt.plot(np.cumsum(rewards), label="Кумулативна награда")
plt.plot(balances, label="Баланс")
plt.title("📈 Представяне на модела върху тестовите данни")
plt.xlabel("Стъпка")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

print(f"📊 Общо сделки: {len(actions)}")
print(f"💰 Финален баланс: {balances[-1]:.2f}")
print(f"📈 Кумулативна награда: {np.sum(rewards):.2f}")


In [ ]:
# Load the data from "/content/eurusd_data.csv"
df = pd.read_csv("/content/eurusd_data.csv", sep=";")

# Convert the 'Time' column to datetime objects and set as index
df["Time"] = pd.to_datetime(df["Time"])
df.set_index("Time", inplace=True)

# Ensure relevant columns are numeric, coercing errors to NaN
for col in ["Open", "High", "Low", "Close", "Volume", "Balance"]:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Recalculate technical indicators and Price Change
# RSI
delta = df["Close"].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
# Handle division by zero for rs
rs = avg_gain / avg_loss.replace(0, np.nan)
df["RSI"] = 100 - (100 / (1 + rs))

# MACD
ema12 = df["Close"].ewm(span=12, adjust=False).mean()
ema26 = df["Close"].ewm(span=26, adjust=False).mean()
df["MACD"] = ema12 - ema26

# Bollinger Bands
rolling_mean = df["Close"].rolling(window=20).mean()
rolling_std = df["Close"].rolling(window=20).std()
df["Bollinger_Middle"] = rolling_mean
df["Bollinger_Upper"] = rolling_mean + (2 * rolling_std)
df["Bollinger_Lower"] = rolling_mean - (2 * rolling_std)

# Price Change
df["Price_Change"] = df["Close"].diff()

# Define the list of 12 features
features = [
    "Open", "High", "Low", "Close", "Volume", "Balance",
    "RSI", "MACD", "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower", "Price_Change"
]

# Split the DataFrame into training and testing sets based on time
train_df = df[df.index < "2024-01-01"].copy()
test_df = df[df.index >= "2024-01-01"].copy()

# Drop rows with NaN values from both sets based on features
train_df.dropna(subset=features, inplace=True)
test_df.dropna(subset=features, inplace=True)

# Convert DataFrames to NumPy arrays
X_train = train_df[features].to_numpy().astype(np.float32)
X_test = test_df[features].to_numpy().astype(np.float32)

# Print the shapes
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

**Reasoning**:
The previous command failed because pandas was not imported. I need to import pandas and re-execute the data loading and processing steps.



In [ ]:
import pandas as pd
import numpy as np

# Load the data from "/content/eurusd_data.csv"
df = pd.read_csv("/content/eurusd_data.csv", sep=";")

# Convert the 'Time' column to datetime objects and set as index
df["Time"] = pd.to_datetime(df["Time"])
df.set_index("Time", inplace=True)

# Ensure relevant columns are numeric, coercing errors to NaN
for col in ["Open", "High", "Low", "Close", "Volume", "Balance"]:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Recalculate technical indicators and Price Change
# RSI
delta = df["Close"].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
# Handle division by zero for rs
rs = avg_gain / avg_loss.replace(0, np.nan)
df["RSI"] = 100 - (100 / (1 + rs))

# MACD
ema12 = df["Close"].ewm(span=12, adjust=False).mean()
ema26 = df["Close"].ewm(span=26, adjust=False).mean()
df["MACD"] = ema12 - ema26

# Bollinger Bands
rolling_mean = df["Close"].rolling(window=20).mean()
rolling_std = df["Close"].rolling(window=20).std()
df["Bollinger_Middle"] = rolling_mean
df["Bollinger_Upper"] = rolling_mean + (2 * rolling_std)
df["Bollinger_Lower"] = rolling_mean - (2 * rolling_std)

# Price Change
df["Price_Change"] = df["Close"].diff()

# Define the list of 12 features
features = [
    "Open", "High", "Low", "Close", "Volume", "Balance",
    "RSI", "MACD", "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower", "Price_Change"
]

# Split the DataFrame into training and testing sets based on time
train_df = df[df.index < "2024-01-01"].copy()
test_df = df[df.index >= "2024-01-01"].copy()

# Drop rows with NaN values from both sets based on features
train_df.dropna(subset=features, inplace=True)
test_df.dropna(subset=features, inplace=True)

# Convert DataFrames to NumPy arrays
X_train = train_df[features].to_numpy().astype(np.float32)
X_test = test_df[features].to_numpy().astype(np.float32)

# Print the shapes
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

**Reasoning**:
The previous command failed because the file '/content/eurusd_data.csv' was not found. I need to check if the file exists in the expected location and if not, locate the correct path and load the data from there. Based on the previous successful execution of cell_id: 7910ff20 which created '/content/eurusd_data_new.csv', I will attempt to load from that file instead.



In [ ]:
import pandas as pd
import numpy as np
import os

# Check if the expected file exists, if not, check the alternative
data_path = "/content/eurusd_data.csv"
if not os.path.exists(data_path):
    data_path = "/content/eurusd_data_new.csv"
    print(f"Original data file not found, using {data_path} instead.")

# Load the data from the correct path
df = pd.read_csv(data_path, sep=";")

# Check if the DataFrame has the expected columns, considering the potential multi-level header from yfinance
# If the first column name is 'Price', it likely has a multi-level header
if df.columns[0] == 'Price':
    # Load again, skipping the first header row
    df = pd.read_csv(data_path, sep=";", header=1)
    # Assign correct column names, assuming the order is consistent
    df.columns = ["Time", "Open", "High", "Low", "Close", "Volume", "Balance", "RSI", "MACD", "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower", "Price_Change"]


# Convert the 'Time' column to datetime objects and set as index
df["Time"] = pd.to_datetime(df["Time"])
df.set_index("Time", inplace=True)

# Ensure relevant columns are numeric, coercing errors to NaN
for col in ["Open", "High", "Low", "Close", "Volume", "Balance", "RSI", "MACD", "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower", "Price_Change"]:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Define the list of 12 features
features = [
    "Open", "High", "Low", "Close", "Volume", "Balance",
    "RSI", "MACD", "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower", "Price_Change"
]

# Split the DataFrame into training and testing sets based on time
train_df = df[df.index < "2024-01-01"].copy()
test_df = df[df.index >= "2024-01-01"].copy()

# Drop rows with NaN values from both sets based on features
train_df.dropna(subset=features, inplace=True)
test_df.dropna(subset=features, inplace=True)

# Convert DataFrames to NumPy arrays
X_train = train_df[features].to_numpy().astype(np.float32)
X_test = test_df[features].to_numpy().astype(np.float32)

# Print the shapes
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

**Reasoning**:
The previous command failed because '/content/eurusd_data_new.csv' was not found either. I need to check the files in the mounted Google Drive folder to find the correct data file name. The `ls` command in cell_id: 4e25c6e5 showed `dtasc_actions_log.csv` and other zip files, but not a general data CSV. Cell cell_id: qSKWqlVXYoLx successfully created 'dtasc_signals.csv' and printed data from 'eurusd_data.csv'. Cell cell_id: 7910ff20 successfully created 'eurusd_data_new.csv' and printed its content. Let's try loading from the path `/content/eurusd_data_new.csv` again, as the traceback indicates it tried to load from there, but might have had an issue with the file state or path. If that fails again, I will check the `/content` directory directly.



In [ ]:
import pandas as pd
import numpy as np
import os

# Define the path to the data file created previously
data_path = "/content/eurusd_data_new.csv"

# Load the data from the correct path
# Added error handling in case the file still isn't found immediately
try:
    df = pd.read_csv(data_path, sep=";")

    # Check if the DataFrame has the expected columns, considering the potential multi-level header from yfinance
    # If the first column name is 'Price', it likely has a multi-level header
    if df.columns[0].startswith('Price'):
        # Load again, skipping the first header row
        print("Detected potential multi-level header, reloading with header=1.")
        df = pd.read_csv(data_path, sep=";", header=1)
        # Assign correct column names, assuming the order is consistent
        # Based on the output of cell 7910ff20, the order should be correct
        df.columns = ["Time", "Open", "High", "Low", "Close", "Volume", "Balance", "RSI", "MACD", "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower", "Price_Change"]
    elif 'Time' not in df.columns:
         # If 'Time' is not a column, assume the first column is Time and there's no multi-level header
         print("Time column not found, assuming first column is Time.")
         df.columns = ["Time"] + df.columns[1:].tolist()


    # Convert the 'Time' column to datetime objects and set as index
    df["Time"] = pd.to_datetime(df["Time"])
    df.set_index("Time", inplace=True)

    # Ensure relevant columns are numeric, coercing errors to NaN
    for col in ["Open", "High", "Low", "Close", "Volume", "Balance", "RSI", "MACD", "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower", "Price_Change"]:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Define the list of 12 features
    features = [
        "Open", "High", "Low", "Close", "Volume", "Balance",
        "RSI", "MACD", "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower", "Price_Change"
    ]

    # Split the DataFrame into training and testing sets based on time
    train_df = df[df.index < "2024-01-01"].copy()
    test_df = df[df.index >= "2024-01-01"].copy()

    # Drop rows with NaN values from both sets based on features
    train_df.dropna(subset=features, inplace=True)
    test_df.dropna(subset=features, inplace=True)

    # Convert DataFrames to NumPy arrays
    X_train = train_df[features].to_numpy().astype(np.float32)
    X_test = test_df[features].to_numpy().astype(np.float32)

    # Print the shapes
    print(f"X_train shape: {X_train.shape}")
    print(f"X_test shape: {X_test.shape}")

except FileNotFoundError:
    print(f"Error: The file {data_path} was not found.")
except Exception as e:
    print(f"An error occurred: {e}")


**Reasoning**:
The data file '/content/eurusd_data_new.csv' was not found in the expected location. I need to check the contents of the `/content` directory to confirm the file name and path before attempting to load it again.



In [ ]:
import os
print(os.listdir('/content'))

**Reasoning**:
The listing of the `/content` directory shows that the data files (`eurusd_data.csv`, `eurusd_data_new.csv`) are not present there. They were likely saved directly to the Google Drive path in the earlier steps. I need to load the data directly from the Google Drive path `/content/drive/MyDrive/DTASC/eurusd_data.csv`, which was created in cell_id: qSKWqlVXYoLx. I will then proceed with the data processing, splitting, and conversion to numpy arrays as per the instructions.



In [ ]:
import pandas as pd
import numpy as np

# Define the path to the data file on Google Drive
data_path = "/content/drive/MyDrive/DTASC/eurusd_data.csv"

# Load the data from the correct path
df = pd.read_csv(data_path, sep=";")

# Convert the 'Time' column to datetime objects and set as index
df["Time"] = pd.to_datetime(df["Time"])
df.set_index("Time", inplace=True)

# Ensure relevant columns are numeric, coercing errors to NaN
for col in ["Open", "High", "Low", "Close", "Volume", "Balance", "RSI", "MACD", "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower"]:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Recalculate technical indicators and Price Change to be safe and consistent
# RSI
delta = df["Close"].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
# Handle division by zero for rs
rs = avg_gain / avg_loss.replace(0, np.nan)
df["RSI"] = 100 - (100 / (1 + rs))

# MACD
ema12 = df["Close"].ewm(span=12, adjust=False).mean()
ema26 = df["Close"].ewm(span=26, adjust=False).mean()
df["MACD"] = ema12 - ema26

# Bollinger Bands
rolling_mean = df["Close"].rolling(window=20).mean()
rolling_std = df["Close"].rolling(window=20).std()
df["Bollinger_Middle"] = rolling_mean
df["Bollinger_Upper"] = rolling_mean + (2 * rolling_std)
df["Bollinger_Lower"] = rolling_mean - (2 * rolling_std)

# Price Change
df["Price_Change"] = df["Close"].diff()


# Define the list of 12 features
features = [
    "Open", "High", "Low", "Close", "Volume", "Balance",
    "RSI", "MACD", "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower", "Price_Change"
]

# Split the DataFrame into training and testing sets based on time
train_df = df[df.index < "2024-01-01"].copy()
test_df = df[df.index >= "2024-01-01"].copy()

# Drop rows with NaN values from both sets based on features
train_df.dropna(subset=features, inplace=True)
test_df.dropna(subset=features, inplace=True)

# Convert DataFrames to NumPy arrays
X_train = train_df[features].to_numpy().astype(np.float32)
X_test = test_df[features].to_numpy().astype(np.float32)

# Print the shapes
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

**Reasoning**:
The previous command failed because '/content/drive/MyDrive/DTASC/eurusd_data.csv' was not found. However, cell_id: 7910ff20 successfully created '/content/eurusd_data_new.csv' and printed its contents and shape, indicating it exists in the /content directory. I will attempt to load from '/content/eurusd_data_new.csv' again, as it seems this is where the data was actually saved. I will include the header handling logic from the previous attempt as the file created by yfinance might have a multi-level header.



In [ ]:
import pandas as pd
import numpy as np

# Define the path to the data file
data_path = "/content/eurusd_data_new.csv"

# Load the data from the correct path
df = pd.read_csv(data_path, sep=";")

# Check if the DataFrame has the expected columns, considering the potential multi-level header from yfinance
# If the first column name is 'Price', it likely has a multi-level header
if df.columns[0].startswith('Price'):
    # Load again, skipping the first header row
    print("Detected potential multi-level header, reloading with header=1.")
    df = pd.read_csv(data_path, sep=";", header=1)
    # Assign correct column names, assuming the order is consistent based on previous outputs
    df.columns = ["Time", "Open", "High", "Low", "Close", "Volume", "Balance", "RSI", "MACD", "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower", "Price_Change"]
elif 'Time' not in df.columns:
     # If 'Time' is not a column, assume the first column is Time and there's no multi-level header
     print("Time column not found, assuming first column is Time.")
     df.columns = ["Time"] + df.columns[1:].tolist()


# Convert the 'Time' column to datetime objects and set as index
df["Time"] = pd.to_datetime(df["Time"])
df.set_index("Time", inplace=True)

# Ensure relevant columns are numeric, coercing errors to NaN
for col in ["Open", "High", "Low", "Close", "Volume", "Balance", "RSI", "MACD", "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower", "Price_Change"]:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Define the list of 12 features
features = [
    "Open", "High", "Low", "Close", "Volume", "Balance",
    "RSI", "MACD", "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower", "Price_Change"
]

# Split the DataFrame into training and testing sets based on time
train_df = df[df.index < "2024-01-01"].copy()
test_df = df[df.index >= "2024-01-01"].copy()

# Drop rows with NaN values from both sets based on features
train_df.dropna(subset=features, inplace=True)
test_df.dropna(subset=features, inplace=True)

# Convert DataFrames to NumPy arrays
X_train = train_df[features].to_numpy().astype(np.float32)
X_test = test_df[features].to_numpy().astype(np.float32)

# Print the shapes
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

**Reasoning**:
The data file '/content/eurusd_data_new.csv' is still not found despite previous attempts and checks. The only other data file mentioned in the history is `eurusd_data.csv`, which was also not found at `/content/drive/MyDrive/DTASC/eurusd_data.csv`. However, cell_id: qSKWqlVXYoLx successfully created `eurusd_data.csv` in the `/content` directory. I will try loading from `/content/eurusd_data.csv` again. I will include the header handling logic as before.



In [ ]:
import pandas as pd
import numpy as np

# Define the path to the data file
data_path = "/content/eurusd_data.csv"

# Load the data from the correct path
df = pd.read_csv(data_path, sep=";")

# Check if the DataFrame has the expected columns, considering the potential multi-level header from yfinance
# If the first column name is 'Price', it likely has a multi-level header
if df.columns[0].startswith('Price'):
    # Load again, skipping the first header row
    print("Detected potential multi-level header, reloading with header=1.")
    df = pd.read_csv(data_path, sep=";", header=1)
    # Assign correct column names, assuming the order is consistent based on previous outputs
    df.columns = ["Time", "Open", "High", "Low", "Close", "Volume", "Balance", "RSI", "MACD", "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower", "Price_Change"]
elif 'Time' not in df.columns:
     # If 'Time' is not a column, assume the first column is Time and there's no multi-level header
     print("Time column not found, assuming first column is Time.")
     df.columns = ["Time"] + df.columns[1:].tolist()


# Convert the 'Time' column to datetime objects and set as index
df["Time"] = pd.to_datetime(df["Time"])
df.set_index("Time", inplace=True)

# Ensure relevant columns are numeric, coercing errors to NaN
for col in ["Open", "High", "Low", "Close", "Volume", "Balance", "RSI", "MACD", "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower", "Price_Change"]:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Define the list of 12 features
features = [
    "Open", "High", "Low", "Close", "Volume", "Balance",
    "RSI", "MACD", "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower", "Price_Change"
]

# Split the DataFrame into training and testing sets based on time
train_df = df[df.index < "2024-01-01"].copy()
test_df = df[df.index >= "2024-01-01"].copy()

# Drop rows with NaN values from both sets based on features
train_df.dropna(subset=features, inplace=True)
test_df.dropna(subset=features, inplace=True)

# Convert DataFrames to NumPy arrays
X_train = train_df[features].to_numpy().astype(np.float32)
X_test = test_df[features].to_numpy().astype(np.float32)

# Print the shapes
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

## Обучение на модела с подобрената среда

### Subtask:
Обучете нов PPO модел, използвайки тренировъчните данни и подобрената `TradingEnv`.


**Reasoning**:
Define the TradingEnv class, create a DummyVecEnv instance with X_train, initialize a PPO model, train the model, and save it.



In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
import gym
from gym import spaces
import numpy as np
import os

# Define the TradingEnv class with realistic reward logic
class TradingEnv(gym.Env):
    def __init__(self, data, initial_balance=100000.0, commission=0.001, stop_loss=0.01, take_profit=0.02, holding_penalty=0.0001):
        super(TradingEnv, self).__init__()
        self.data = data
        self.initial_balance = initial_balance
        self.commission = commission
        self.stop_loss = stop_loss
        self.take_profit = take_profit
        self.holding_penalty = holding_penalty

        self.action_space = spaces.Discrete(2)  # 0 = SELL, 1 = BUY
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(data.shape[1],), dtype=np.float32)

        self.reset()

    def reset(self):
        self.current_step = 0
        self.balance = self.initial_balance
        self.position = 0  # 0 = no position, 1 = long
        self.entry_price = 0.0
        self.holding_steps = 0
        return self.data[self.current_step]

    def step(self, action):
        self.current_step += 1
        done = self.current_step >= len(self.data) - 1
        obs = self.data[self.current_step] if not done else np.zeros_like(self.data[0])
        price = self.data[self.current_step][3]  # Close

        reward = 0.0

        if action == 1 and self.position == 0:
            # BUY
            self.position = 1
            self.entry_price = price
            self.holding_steps = 0
            self.balance -= price * self.commission

        elif action == 0 and self.position == 1:
            # SELL
            profit = price - self.entry_price
            reward = profit - price * self.commission
            self.balance += profit
            self.position = 0
            self.entry_price = 0.0

        elif self.position == 1:
            # HOLD
            self.holding_steps += 1
            unrealized = price - self.entry_price

            # Stop-loss
            if unrealized <= -self.stop_loss * self.entry_price:
                reward = unrealized - price * self.commission
                self.balance += unrealized
                self.position = 0
                self.entry_price = 0.0

            # Take-profit
            elif unrealized >= self.take_profit * self.entry_price:
                reward = unrealized - price * self.commission
                self.balance += unrealized
                self.position = 0
                self.entry_price = 0.0

            else:
                reward -= self.holding_penalty * self.holding_steps

        return obs, reward, done, {}

    def render(self, mode="human"):
        print(f"Step: {self.current_step}, Balance: {self.balance:.2f}, Position: {self.position}, Entry: {self.entry_price:.5f}")


# Create DummyVecEnv with the improved TradingEnv and X_train data
env_train = DummyVecEnv([lambda: TradingEnv(X_train)])

# Initialize PPO model
model = PPO("MlpPolicy", env_train, verbose=1)

# Train the model
model.learn(total_timesteps=10000)

# Save the trained model
model_save_path = "/content/drive/MyDrive/DTASC/ppo_model_trained_on_train_df.zip"
model.save(model_save_path)

print(f"✅ Моделът е обучен и запазен като '{model_save_path}'")

**Reasoning**:
The previous command failed because the stable_baselines3 library was not found. I need to reinstall it.



In [ ]:
!pip install stable-baselines3

**Reasoning**:
Now that stable_baselines3 is installed, I can proceed with defining the TradingEnv, creating the training environment, initializing and training the PPO model, and saving it as outlined in the subtask instructions.



In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
import gym
from gym import spaces
import numpy as np
import os

# Define the TradingEnv class with realistic reward logic
class TradingEnv(gym.Env):
    def __init__(self, data, initial_balance=100000.0, commission=0.001, stop_loss=0.01, take_profit=0.02, holding_penalty=0.0001):
        super(TradingEnv, self).__init__()
        self.data = data
        self.initial_balance = initial_balance
        self.commission = commission
        self.stop_loss = stop_loss
        self.take_profit = take_profit
        self.holding_penalty = holding_penalty

        self.action_space = spaces.Discrete(2)  # 0 = SELL, 1 = BUY
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(data.shape[1],), dtype=np.float32)

        self.reset()

    def reset(self):
        self.current_step = 0
        self.balance = self.initial_balance
        self.position = 0  # 0 = no position, 1 = long
        self.entry_price = 0.0
        self.holding_steps = 0
        return self.data[self.current_step]

    def step(self, action):
        self.current_step += 1
        done = self.current_step >= len(self.data) - 1
        obs = self.data[self.current_step] if not done else np.zeros_like(self.data[0])
        price = self.data[self.current_step][3]  # Close

        reward = 0.0

        if action == 1 and self.position == 0:
            # BUY
            self.position = 1
            self.entry_price = price
            self.holding_steps = 0
            self.balance -= price * self.commission

        elif action == 0 and self.position == 1:
            # SELL
            profit = price - self.entry_price
            reward = profit - price * self.commission
            self.balance += profit
            self.position = 0
            self.entry_price = 0.0

        elif self.position == 1:
            # HOLD
            self.holding_steps += 1
            unrealized = price - self.entry_price

            # Stop-loss
            if unrealized <= -self.stop_loss * self.entry_price:
                reward = unrealized - price * self.commission
                self.balance += unrealized
                self.position = 0
                self.entry_price = 0.0

            # Take-profit
            elif unrealized >= self.take_profit * self.entry_price:
                reward = unrealized - price * self.commission
                self.balance += unrealized
                self.position = 0
                self.entry_price = 0.0

            else:
                reward -= self.holding_penalty * self.holding_steps

        return obs, reward, done, {}

    def render(self, mode="human"):
        print(f"Step: {self.current_step}, Balance: {self.balance:.2f}, Position: {self.position}, Entry: {self.entry_price:.5f}")


# Create DummyVecEnv with the improved TradingEnv and X_train data
env_train = DummyVecEnv([lambda: TradingEnv(X_train)])

# Initialize PPO model
model = PPO("MlpPolicy", env_train, verbose=1)

# Train the model
model.learn(total_timesteps=10000)

# Save the trained model
model_save_path = "/content/drive/MyDrive/DTASC/ppo_model_trained_on_train_df.zip"
model.save(model_save_path)

print(f"✅ Моделът е обучен и запазен като '{model_save_path}'")

In [ ]:
import pandas as pd
import numpy as np
import os

# Define the path to the data file on Google Drive
data_path = "/content/drive/MyDrive/DTASC/eurusd_data.csv"

# Load the data from the correct path
try:
    df = pd.read_csv(data_path, sep=";")

    # Check if the DataFrame has the expected columns, considering the potential multi-level header from yfinance
    # If the first column name is 'Price', it likely has a multi-level header
    if df.columns[0].startswith('Price'):
        # Load again, skipping the first header row
        print("Detected potential multi-level header, reloading with header=1.")
        df = pd.read_csv(data_path, sep=";", header=1)
        # Assign correct column names, assuming the order is consistent based on previous outputs
        df.columns = ["Time", "Open", "High", "Low", "Close", "Volume", "Balance", "RSI", "MACD", "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower", "Price_Change"]
    elif 'Time' not in df.columns:
         # If 'Time' is not a column, assume the first column is Time and there's no multi-level header
         print("Time column not found, assuming first column is Time.")
         df.columns = ["Time"] + df.columns[1:].tolist()


    # Convert the 'Time' column to datetime objects and set as index
    df["Time"] = pd.to_datetime(df["Time"])
    df.set_index("Time", inplace=True)

    # Ensure relevant columns are numeric, coercing errors to NaN
    for col in ["Open", "High", "Low", "Close", "Volume", "Balance", "RSI", "MACD", "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower", "Price_Change"]:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Define the list of 12 features
    features = [
        "Open", "High", "Low", "Close", "Volume", "Balance",
        "RSI", "MACD", "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower", "Price_Change"
    ]

    # Split the DataFrame into training and testing sets based on time
    train_df = df[df.index < "2024-01-01"].copy()
    test_df = df[df.index >= "2024-01-01"].copy()

    # Drop rows with NaN values from both sets based on features
    train_df.dropna(subset=features, inplace=True)
    test_df.dropna(subset=features, inplace=True)

    # Convert DataFrames to NumPy arrays
    X_train = train_df[features].to_numpy().astype(np.float32)
    X_test = test_df[features].to_numpy().astype(np.float32)

    # Print the shapes
    print(f"X_train shape: {X_train.shape}")
    print(f"X_test shape: {X_test.shape}")

except FileNotFoundError:
    print(f"Error: The data file {data_path} was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
import gym
from gym import spaces
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt

# Define the TradingEnv class with realistic reward logic (copied from cell_id: d64bbc68)
class TradingEnv(gym.Env):
    def __init__(self, data, initial_balance=100000.0, commission=0.001, stop_loss=0.01, take_profit=0.02, holding_penalty=0.0001):
        super(TradingEnv, self).__init__()
        self.data = data
        self.initial_balance = initial_balance
        self.commission = commission
        self.stop_loss = stop_loss
        self.take_profit = take_profit
        self.holding_penalty = holding_penalty

        self.action_space = spaces.Discrete(2)  # 0 = SELL, 1 = BUY
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(data.shape[1],), dtype=np.float32)

        self.reset()

    def reset(self):
        self.current_step = 0
        self.balance = self.initial_balance
        self.position = 0  # 0 = no position, 1 = long
        self.entry_price = 0.0
        self.holding_steps = 0
        return self.data[self.current_step]

    def step(self, action):
        self.current_step += 1
        done = self.current_step >= len(self.data) - 1
        obs = self.data[self.current_step] if not done else np.zeros_like(self.data[0])
        price = self.data[self.current_step][3]  # Close

        reward = 0.0

        if action == 1 and self.position == 0:
            # BUY
            self.position = 1
            self.entry_price = price
            self.holding_steps = 0
            self.balance -= price * self.commission

        elif action == 0 and self.position == 1:
            # SELL
            profit = price - self.entry_price
            reward = profit - price * self.commission
            self.balance += profit
            self.position = 0
            self.entry_price = 0.0

        elif self.position == 1:
            # HOLD
            self.holding_steps += 1
            unrealized = price - self.entry_price

            # Stop-loss
            if unrealized <= -self.stop_loss * self.entry_price:
                reward = unrealized - price * self.commission
                self.balance += unrealized
                self.position = 0
                self.entry_price = 0.0

            # Take-profit
            elif unrealized >= self.take_profit * self.entry_price:
                reward = unrealized - price * self.commission
                self.balance += unrealized
                self.position = 0
                self.entry_price = 0.0

            else:
                reward -= self.holding_penalty * self.holding_steps

        return obs, reward, done, {}

    def render(self, mode="human"):
        print(f"Step: {self.current_step}, Balance: {self.balance:.2f}, Position: {self.position}, Entry: {self.entry_price:.5f}")


# --- Model Training ---
print("\nTraining the model...")
env_train = DummyVecEnv([lambda: TradingEnv(X_train)])
model = PPO("MlpPolicy", env_train, verbose=1)
model.learn(total_timesteps=10000) # You can adjust total_timesteps as needed

# Save the trained model
model_save_path = "/content/drive/MyDrive/DTASC/ppo_model_trained_on_train_df.zip"
model.save(model_save_path)
print(f"✅ Моделът е обучен и запазен като '{model_save_path}'")


# --- Model Evaluation ---
print("\nEvaluating the model on test data...")
# Load the trained model
model = PPO.load(model_save_path)

# Simulation on test data
env_test = TradingEnv(X_test)
obs = env_test.reset()
rewards = []
balance = env_test.initial_balance # Use the initial balance from the environment
equity = [balance] # Start equity with initial balance
position = 0 # Track position for equity calculation
entry_price = 0.0 # Track entry price for equity calculation

for i in range(len(X_test) - 1):
    action, _ = model.predict(obs.reshape(1, -1), deterministic=True)
    action = int(action[0]) if isinstance(action, np.ndarray) else int(action) # Ensure action is integer

    obs, reward, done, info = env_test.step(action) # Get info dictionary

    rewards.append(reward)

    # Update position and entry price based on action for equity calculation
    current_price = X_test[i+1][3] # Close price of the next step
    if action == 1 and position == 0: # BUY
        position = 1
        entry_price = current_price # Entry price is the price of the step *after* the BUY action
    elif action == 0 and position == 1: # SELL
        position = 0
        entry_price = 0.0 # Reset entry price on SELL

    # Calculate current equity: balance + profit/loss from current position
    current_equity = env_test.balance + (X_test[i+1][3] - env_test.entry_price if env_test.position else 0)
    equity.append(current_equity)


    if done:
        break

# Visualization
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(np.cumsum(rewards), label="Кумулативна награда")
plt.title("📊 Оценка на модела върху тестови данни (Кумулативна награда)")
plt.xlabel("Стъпка")
plt.ylabel("Награда")
plt.grid()
plt.legend()

plt.subplot(1, 2, 2)
# Plot equity against the length of equity list
plt.plot(equity, label="Баланс (Equity)")
plt.title("💰 Оценка на модела върху тестови данни (Баланс)")
plt.xlabel("Стъпка")
plt.ylabel("Баланс")
plt.grid()
plt.legend()

plt.tight_layout()
plt.show()

# Calculate and print trading metrics
total_reward = np.sum(rewards)
final_balance = equity[-1] if equity else env_test.initial_balance
initial_balance = env_test.initial_balance
profit_loss = final_balance - initial_balance

print("\n--- Търговски метрики върху тестови данни ---")
print(f"Начален баланс: {initial_balance:.2f}")
print(f"Краен баланс: {final_balance:.2f}")
print(f"Обща печалба/загуба: {profit_loss:.2f}")
print(f"Обща кумулативна награда: {total_reward:.2f}")
print("--------------------------------------------")

In [ ]:
import pandas as pd
import numpy as np
from stable_baselines3 import PPO
import os

# Define the path to the trained model
model_path = "/content/drive/MyDrive/DTASC/ppo_model_trained_on_train_df.zip"

# Define the path for the action log file
log_path = "/content/drive/MyDrive/DTASC/dtasc_actions_log.csv"

# Load the trained model
try:
    model = PPO.load(model_path)
    print(f"✅ Моделът е успешно зареден от '{model_path}'")
except FileNotFoundError:
    print(f"❌ Грешка: Моделът не е намерен на '{model_path}'. Моля, уверете се, че файлът съществува.")
    # Exit the cell if the model is not found
    raise
except Exception as e:
    print(f"❌ Грешка при зареждане на модела: {e}")
    # Exit the cell if model loading fails
    raise


# Ensure X_test is available from previous steps
if 'X_test' not in locals():
    print("❌ Грешка: X_test не е наличен. Моля, изпълнете стъпките за подготовка на данните.")
    # Exit the cell if X_test is not available
    raise NameError("X_test is not defined")

# Assuming test_df is also available with the 'Time' index
if 'test_df' not in locals():
     print("❌ Грешка: test_df не е наличен. Моля, изпълнете стъпките за подготовка на данните.")
     # Exit the cell if test_df is not available
     raise NameError("test_df is not defined")


# Simulation and Logging
log = []
balance = 100000.0 # Initial balance
position = 0 # 0 = no position, 1 = long
entry_price = 0.0
commission = 0.001

print("\n🚀 Стартира симулацията и логването...")

# Iterate through the test data
# Start from the second data point to have a price change from the previous step
for i in range(len(X_test)):
    obs = X_test[i].reshape(1, -1)
    action, _ = model.predict(obs, deterministic=True)
    # Ensure action is an integer
    action = int(action[0]) if isinstance(action, np.ndarray) else int(action)

    price = X_test[i][3]  # Close price is the 4th feature (index 3)

    reward = 0.0
    note = "HOLD" # Default action note

    # Determine action and calculate reward
    if action == 1 and position == 0:  # BUY
        position = 1
        entry_price = price
        balance -= price * commission
        note = "BUY"
    elif action == 0 and position == 1:  # SELL
        profit = price - entry_price
        reward = profit - price * commission
        balance += profit
        position = 0
        entry_price = 0.0
        note = "SELL"

    # Log the action and state
    # Use the index from the original test_df for the timestamp
    if i < len(test_df.index):
         timestamp = test_df.index[i]
    else:
         # If somehow i is out of bounds for test_df index, use a placeholder or the last available index
         timestamp = test_df.index[-1] if not test_df.empty else f"Step_{i}"


    log.append({
        "Time": timestamp,
        "Action": note,
        "Price": price,
        "Balance": balance,
        "Reward": reward,
        "Position": position
    })

print(f"🏁 Симулацията завърши. Логване на {len(log)} стъпки.")

# Save the log to CSV
log_df = pd.DataFrame(log)
log_df.to_csv(log_path, index=False, sep=";")
print(f"✅ Записан лог файл: '{log_path}'")

# Optional: Display the first few rows of the log
print("\n📋 Първи 5 реда от лог файла:\n")
display(log_df.head())

# Optional: Display the last few rows of the log
print("\n📋 Последни 5 реда от лог файла:\n")
display(log_df.tail())

In [ ]:
log = []
for step in range(len(X)):
    action, _ = model.predict(X[step].reshape(1, -1), deterministic=True)
    log.append({"Step": step, "Action": int(action[0]), "Price": df["Close"].iloc[step]})
pd.DataFrame(log).to_csv("dtasc_actions_log.csv", sep=";", index=False)


## Логване на действията на модела

### Subtask:
Добавете функционалност за логване на действията на модела (BUY/SELL/HOLD), цената, баланса, наградата и позицията във всяка стъпка от симулацията върху тестовите данни в CSV файл.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Define the path to the action log file
log_path = "/content/drive/MyDrive/DTASC/dtasc_actions_log.csv"

# Load the action log file
try:
    log_df = pd.read_csv(log_path, sep=";")
    print(f"✅ Лог файлът е успешно зареден от '{log_path}'")
except FileNotFoundError:
    print(f"❌ Грешка: Лог файлът не е намерен на '{log_path}'. Моля, уверете се, че файлът съществува.")
    raise
except Exception as e:
    print(f"❌ Грешка при зареждане на лог файла: {e}")
    raise


# Convert the 'Time' column to datetime objects for proper plotting
log_df["Time"] = pd.to_datetime(log_df["Time"])
log_df.set_index("Time", inplace=True)

# Plot the Close Price from the log data
plt.figure(figsize=(12, 6))
plt.plot(log_df.index, log_df["Price"].values, label="Close Price", color="black")

# Filter BUY and SELL signals from the log data
buy_signals = log_df[log_df["Action"] == "BUY"]
sell_signals = log_df[log_df["Action"] == "SELL"]

# Plot BUY signals
plt.scatter(buy_signals.index, buy_signals["Price"], marker="^", color="green", label="BUY", alpha=1)

# Plot SELL signals
plt.scatter(sell_signals.index, sell_signals["Price"], marker="v", color="red", label="SELL", alpha=1)

plt.title("📈 BUY/SELL сигнали от модела върху тестови данни")
plt.xlabel("Време")
plt.ylabel("Цена")
plt.legend()
plt.grid()
plt.show()

In [ ]:
from stable_baselines3 import A2C, DQN
model = A2C("MlpPolicy", env, verbose=1)


In [ ]:
model = PPO("MlpPolicy", env, learning_rate=0.0003, n_steps=256, gamma=0.99, ent_coef=0.01, verbose=1)


**Поправка на грешката:**

In [ ]:
# Original error code:
# if action == 1:  # BUY
#     entry_price = self.data[self.current_step][3]  # Close
# elif action == 0 and position_open:
#     exit_price = self.data[self.current_step][3]
#     reward = (exit_price - entry_price) - commission

# Corrected code assuming it's within the TradingEnv step method:
# Note: This is a conceptual fix based on the error.
# The actual implementation in the TradingEnv step method (cell_id: d64bbc68) is already more complete.

# Example of how to fix the NameError in the snippet:
# if action == 1 and self.position == 0: # Check if can BUY (no position)
#     self.position = 1
#     self.entry_price = self.data[self.current_step][3] # Close price
#     # Handle commission here if needed for opening
# elif action == 0 and self.position == 1: # Check if can SELL (position open)
#     exit_price = self.data[self.current_step][3] # Close price
#     profit = exit_price - self.entry_price
#     reward = profit # Calculate reward based on profit
#     # Handle commission here if needed for closing
#     self.balance += reward # Update balance based on profit/loss (including commission if handled in reward)
#     self.position = 0
#     self.entry_price = 0.0
# else:
#     # Handle HOLD action or invalid actions
#     pass # Or apply holding penalty etc.


# The step method in cell_id: d64bbc68 already contains a more robust implementation.
# No changes are needed to the existing TradingEnv class as the error was in a separate snippet.

print("✅ Грешката е обяснена и предложено решение. Съществуващият TradingEnv клас вече има по-цялостна логика.")

In [ ]:
df["TR"] = np.maximum(df["High"] - df["Low"],
                      np.maximum(abs(df["High"] - df["Close"].shift(1)),
                                 abs(df["Low"] - df["Close"].shift(1))))
df["ATR"] = df["TR"].rolling(window=14).mean()


In [ ]:
df["Pivot"] = (df["High"] + df["Low"] + df["Close"]) / 3
df["R1"] = 2 * df["Pivot"] - df["Low"]
df["S1"] = 2 * df["Pivot"] - df["High"]


In [ ]:
# ATR (Average True Range)
df["TR"] = np.maximum(df["High"] - df["Low"],
                      np.maximum(abs(df["High"] - df["Close"].shift(1)),
                                 abs(df["Low"] - df["Close"].shift(1))))
df["ATR"] = df["TR"].rolling(window=14).mean()

# Pivot Point + R1/S1
df["Pivot"] = (df["High"] + df["Low"] + df["Close"]) / 3
df["R1"] = 2 * df["Pivot"] - df["Low"]
df["S1"] = 2 * df["Pivot"] - df["High"]


In [ ]:
features = [
    "Open", "High", "Low", "Close", "Volume", "RSI", "MACD",
    "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower",
    "Balance", "Price_Change", "ATR", "Pivot", "R1", "S1"
]


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

# 🔁 Изтегляне на данни
df = yf.download("EURUSD=X", interval="15m", period="1d")

# Flatten the multi-level column index
# Check if the columns have a MultiIndex (which happens for intervals < 1d)
if isinstance(df.columns, pd.MultiIndex):
    # Create new column names by joining the levels
    df.columns = ['_'.join(col).strip() for col in df.columns.values]
    # Print columns after flattening to diagnose
    print("Columns after flattening:", df.columns.tolist())

    # Rename specific columns to match the expected feature names
    df.rename(columns={
        'Open_EURUSD=X': 'Open',
        'High_EURUSD=X': 'High',
        'Low_EURUSD=X': 'Low',
        'Close_EURUSD=X': 'Close',
        'Volume_EURUSD=X': 'Volume'
    }, inplace=True)
    # Drop the Ticker level columns if they still exist and are not needed
    df = df.loc[:, ~df.columns.str.startswith('Ticker')]

# Add print statements to diagnose after renaming
print("Columns after renaming:", df.columns.tolist())
print("DataFrame head after renaming:\n", df.head())

# Ensure standard columns exist after renaming - keep this check for safety
required_cols = ["Open", "High", "Low", "Close", "Volume"]
missing_cols = [col for col in required_cols if col not in df.columns]
if missing_cols:
    raise KeyError(f"Missing required columns after flattening and renaming: {missing_cols}")


# 📈 RSI
delta = df["Close"].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
# Handle division by zero for rs
# Check if avg_loss is zero or contains NaNs before division
if avg_loss.isnull().all() or (avg_loss == 0).all():
    rs = pd.Series(np.nan, index=df.index) # Avoid division by zero if avg_loss is all zero or NaN
else:
    rs = avg_gain / avg_loss.replace(0, np.nan) # Replace zero with NaN to handle division by zero

df["RSI"] = 100 - (100 / (1 + rs))

# 📉 MACD
ema12 = df["Close"].ewm(span=12, adjust=False).mean()
ema26 = df["Close"].ewm(span=26, adjust=False).mean()
df["MACD"] = ema12 - ema26

# 📊 Bollinger Bands
rolling_mean = df["Close"].rolling(window=20).mean()
rolling_std = df["Close"].rolling(window=20).std()
df["Bollinger_Middle"] = rolling_mean
df["Bollinger_Upper"] = rolling_mean + 2 * rolling_std
df["Bollinger_Lower"] = rolling_mean - 2 * rolling_std

# 🔁 Price Change
# Calculate Price Change *after* ensuring 'Close' is a single column
df["Price_Change"] = df["Close"].pct_change()

# 📏 ATR (Average True Range)
df["TR"] = np.maximum(df["High"] - df["Low"],
                      np.maximum(abs(df["High"] - df["Close"].shift(1)),
                                 abs(df["Low"] - df["Close"].shift(1))))
df["ATR"] = df["TR"].rolling(window=14).mean()

# 📌 Pivot Points
df["Pivot"] = (df["High"] + df["Low"] + df["Close"]) / 3
df["R1"] = 2 * df["Pivot"] - df["Low"]
df["S1"] = 2 * df["Pivot"] - df["High"]

# 💰 Фиктивен баланс
df["Balance"] = 100000.0

# ✅ Записване на пълните входове
features = [
    "Open", "High", "Low", "Close", "Volume", "RSI", "MACD",
    "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower",
    "Balance", "Price_Change", "ATR", "Pivot", "R1", "S1"
]

# Select the features and then drop NaNs
df = df[features].dropna().copy() # Add .copy() to avoid SettingWithCopyWarning

print(f"✅ X_test.csv обновен ({len(df)} реда) с {len(features)} входа")

# Save the processed data
df.to_csv("/content/drive/MyDrive/DTASC/X_test.csv", sep=";", index=False)

In [ ]:
import pandas as pd
import numpy as np
import gym
from gym import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

# 1️⃣ Зареждане на данните
df = pd.read_csv("/content/eurusd_data.csv", sep=";")

# Convert relevant columns to numeric, coercing errors to NaN
for col in ["Open", "High", "Low", "Close", "Volume", "Balance", "RSI", "MACD", "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower"]:
    df[col] = pd.to_numeric(df[col], errors='coerce')


# 2️⃣ Изчисляване на липсващите входни характеристики
# Price Change
df["Price_Change"] = df["Close"].diff()

# ATR (Average True Range)
df["TR"] = np.maximum(df["High"] - df["Low"],
                      np.maximum(abs(df["High"] - df["Close"].shift(1)),
                                 abs(df["Low"] - df["Close"].shift(1))))
df["ATR"] = df["TR"].rolling(window=14).mean()

# Pivot Point + R1/S1
df["Pivot"] = (df["High"] + df["Low"] + df["Close"]) / 3
df["R1"] = 2 * df["Pivot"] - df["Low"]
df["S1"] = 2 * df["Pivot"] - df["High"]


# 3️⃣ Подготовка на входните характеристики (включително новоизчислените)
features = [
    "Open", "High", "Low", "Close", "Volume", "RSI", "MACD",
    "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower",
    "Balance", "Price_Change", "ATR", "Pivot", "R1", "S1"
]

# Drop rows with NaN values that resulted from calculating indicators
df.dropna(subset=features, inplace=True)

X = df[features].to_numpy().astype(np.float32)

# 4️⃣ Дефиниране на средата (използвайки вече дефинирания клас TradingEnv, ако е наличен)
# Ако TradingEnv не е дефиниран в тази клетка или преди това, трябва да го дефинираме тук
# Assuming TradingEnv is defined elsewhere (e.g., cell_id: d64bbc68 or h2OuY7yk2bQ5)
# If not, uncomment and use the definition from cell_id: d64bbc68

# class TradingEnv(gym.Env):
#     def __init__(self, data, initial_balance=100000.0, commission=0.001, stop_loss=0.01, take_profit=0.02, holding_penalty=0.0001):
#         super(TradingEnv, self).__init__()
#         self.data = data
#         self.initial_balance = initial_balance
#         self.commission = commission
#         self.stop_loss = stop_loss
#         self.take_profit = take_profit
#         self.holding_penalty = holding_penalty

#         self.action_space = spaces.Discrete(2)  # 0 = SELL, 1 = BUY
#         self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(data.shape[1],), dtype=np.float32)

#         self.reset()

#     def reset(self):
#         self.current_step = 0
#         self.balance = self.initial_balance
#         self.position = 0  # 0 = no position, 1 = long
#         self.entry_price = 0.0
#         self.holding_steps = 0
#         return self.data[self.current_step]

#     def step(self, action):
#         self.current_step += 1
#         done = self.current_step >= len(self.data) - 1
#         obs = self.data[self.current_step] if not done else np.zeros_like(self.data[0])
#         price = self.data[self.current_step][3]  # Close

#         reward = 0.0

#         if action == 1 and self.position == 0:
#             # BUY
#             self.position = 1
#             self.entry_price = price
#             self.holding_steps = 0
#             self.balance -= price * self.commission

#         elif action == 0 and self.position == 1:
#             # SELL
#             profit = price - self.entry_price
#             reward = profit - price * self.commission
#             self.balance += profit
#             self.position = 0
#             self.entry_price = 0.0

#         elif self.position == 1:
#             # HOLD
#             self.holding_steps += 1
#             unrealized = price - self.entry_price

#             # Stop-loss
#             if unrealized <= -self.stop_loss * self.entry_price:
#                 reward = unrealized - price * self.commission
#                 self.balance += unrealized
#                 self.position = 0
#                 self.entry_price = 0.0

#             # Take-profit
#             elif unrealized >= self.take_profit * self.entry_price:
#                 reward = unrealized - price * self.commission
#                 self.balance += unrealized
#                 self.position = 0
#                 self.entry_price = 0.0

#             else:
#                 reward -= self.holding_penalty * self.holding_steps

#         return obs, reward, done, {}

#     def render(self, mode="human"):
#         print(f"Step: {self.current_step}, Balance: {self.balance:.2f}, Position: {self.position}, Entry: {self.entry_price:.5f}")


# 5️⃣ Обучение на модела
env = DummyVecEnv([lambda: TradingEnv(X)])
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)

# 6️⃣ Запазване на модела
model.save("/content/drive/MyDrive/DTASC/model_retrain_latest.zip")
print("✅ Моделът е успешно обучен и запазен като model_retrain_latest.zip")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from stable_baselines3 import PPO

# Зареждане на данните
df = pd.read_csv("/content/eurusd_data.csv", sep=";")

# Convert relevant columns to numeric, coercing errors to NaN
# Ensure all potentially numeric columns, including indicators from the file, are converted
for col in ["Open", "High", "Low", "Close", "Volume", "Balance", "RSI", "MACD", "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower"]:
    df[col] = pd.to_numeric(df[col], errors='coerce')


# Изчисляване на липсващите входни характеристики (Price_Change, ATR, Pivot, R1, S1)
# Price Change
df["Price_Change"] = df["Close"].diff()

# ATR (Average True Range)
df["TR"] = np.maximum(df["High"] - df["Low"],
                      np.maximum(abs(df["High"] - df["Close"].shift(1)),
                                 abs(df["Low"] - df["Close"].shift(1))))
df["ATR"] = df["TR"].rolling(window=14).mean()

# Pivot Point + R1/S1
df["Pivot"] = (df["High"] + df["Low"] + df["Close"]) / 3
df["R1"] = 2 * df["Pivot"] - df["Low"]
df["S1"] = 2 * df["Pivot"] - df["High"]


features = [
    "Open", "High", "Low", "Close", "Volume", "RSI", "MACD",
    "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower",
    "Balance", "Price_Change", "ATR", "Pivot", "R1", "S1"
]

# Drop rows with NaN values that resulted from calculating indicators
df.dropna(subset=features, inplace=True)

X = df[features].to_numpy().astype(np.float32)

# Зареждане на модела
# Use the correct model file path if needed, assuming model_retrain_latest.zip is the target
model_path = "/content/drive/MyDrive/DTASC/model_retrain_latest.zip"
try:
    model = PPO.load(model_path)
    print(f"✅ Моделът е успешно зареден от '{model_path}'")
except FileNotFoundError:
    print(f"❌ Грешка: Моделът не е намерен на '{model_path}'. Моля, уверете се, че файлът съществува.")
    raise
except Exception as e:
    print(f"❌ Грешка при зареждане на модела: {e}")
    raise


# Предсказване на действия
actions = []
# Iterate through the data (which is now X after dropping NaNs)
for i in range(len(X)):
    # Reshape observation for prediction (batch size 1)
    obs = X[i].reshape(1, -1)
    action, _ = model.predict(obs, deterministic=True)
    # Ensure action is an integer
    action = int(action[0]) if isinstance(action, np.ndarray) else int(action)
    actions.append(action)

# Add the predicted actions to the DataFrame (after dropping NaNs to match X)
# Ensure the index is aligned. Since X is derived from df after dropna, use df.index
df["Action"] = actions

# Визуализация
plt.figure(figsize=(14, 6))
# Use df.index for plotting the price against time
plt.plot(df.index, df["Close"].values, label="Цена", linewidth=1.5)

buy_signals = df[df["Action"] == 1]
sell_signals = df[df["Action"] == 0]

plt.scatter(buy_signals.index, buy_signals["Close"], marker="^", color="green", label="BUY", s=80)
plt.scatter(sell_signals.index, sell_signals["Close"], marker="v", color="red", label="SELL", s=80)

plt.title("📈 BUY/SELL сигнали от модела върху Close цената")
plt.xlabel("Индекс")
plt.ylabel("Цена")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
plt.savefig("/content/drive/MyDrive/DTASC/model_signals_plot.png", dpi=300)


In [ ]:
# Изчисляване на метрики
total_trades = len(df) - 1
buy_trades = df["Action"].sum()
sell_trades = total_trades - buy_trades

# Примерна печалба/загуба (фиктивна логика)
df["PnL"] = df["Close"].diff() * df["Action"].shift(1).fillna(0)
cumulative_reward = df["PnL"].cumsum()
sharpe_ratio = df["PnL"].mean() / df["PnL"].std() * np.sqrt(252) if df["PnL"].std() != 0 else 0
max_drawdown = (cumulative_reward.cummax() - cumulative_reward).max()

print(f"📈 Общо сделки: {total_trades}")
print(f"🟢 BUY сделки: {buy_trades}")
print(f"🔴 SELL сделки: {sell_trades}")
print(f"💰 Кумулативна печалба: {cumulative_reward.iloc[-1]:.2f}")
print(f"📉 Max Drawdown: {max_drawdown:.2f}")
print(f"📊 Sharpe Ratio: {sharpe_ratio:.2f}")


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from stable_baselines3 import PPO

# 1️⃣ Зареждане на данните
df = pd.read_csv("/content/eurusd_data.csv", sep=";")

# Convert relevant columns to numeric, coercing errors to NaN
# Ensure all potentially numeric columns, including indicators from the file, are converted
for col in ["Open", "High", "Low", "Close", "Volume", "Balance", "RSI", "MACD", "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower"]:
    df[col] = pd.to_numeric(df[col], errors='coerce')


# Изчисляване на липсващите входни характеристики (Price_Change, ATR, Pivot, R1, S1)
# Price Change
df["Price_Change"] = df["Close"].diff()

# ATR (Average True Range)
df["TR"] = np.maximum(df["High"] - df["Low"],
                      np.maximum(abs(df["High"] - df["Close"].shift(1)),
                                 abs(df["Low"] - df["Close"].shift(1))))
df["ATR"] = df["TR"].rolling(window=14).mean()

# Pivot Point + R1/S1
df["Pivot"] = (df["High"] + df["Low"] + df["Close"]) / 3
df["R1"] = 2 * df["Pivot"] - df["Low"]
df["S1"] = 2 * df["Pivot"] - df["High"]


features = [
    "Open", "High", "Low", "Close", "Volume", "RSI", "MACD",
    "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower",
    "Balance", "Price_Change", "ATR", "Pivot", "R1", "S1"
]

# Drop rows with NaN values that resulted from calculating indicators
df.dropna(subset=features, inplace=True)

X = df[features].to_numpy().astype(np.float32)

# 2️⃣ Зареждане на модела
model = PPO.load("/content/drive/MyDrive/DTASC/model_retrain_latest.zip")

# 3️⃣ Симулация на действия
actions = []
pnl = []
position = None
entry_price = 0

for i in range(len(X)):
    action, _ = model.predict(X[i].reshape(1, -1), deterministic=True)
    action = int(action[0])
    actions.append(action)

    price = df["Close"].iloc[i]
    if action == 1 and position is None:
        position = "long"
        entry_price = price
        pnl.append(0)
    elif action == 0 and position == "long":
        profit = price - entry_price
        pnl.append(profit)
        position = None
    else:
        pnl.append(0)

df["Action"] = actions
df["PnL"] = pnl
df["Cumulative_PnL"] = df["PnL"].cumsum()

# 4️⃣ Визуализация
plt.figure(figsize=(14, 6))
plt.plot(df["Close"].values, label="Цена", linewidth=1.5)
plt.scatter(df[df["Action"] == 1].index, df[df["Action"] == 1]["Close"], marker="^", color="green", label="BUY", s=80)
plt.scatter(df[df["Action"] == 0].index, df[df["Action"] == 0]["Close"], marker="v", color="red", label="SELL", s=80)
plt.title("📈 BUY/SELL сигнали от модела")
plt.xlabel("Индекс")
plt.ylabel("Цена")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("/content/drive/MyDrive/DTASC/model_signals_plot.png", dpi=300)
plt.show()

# 5️⃣ Метрики
total_trades = df["Action"].diff().abs().sum()
successful_trades = df["PnL"][df["PnL"] > 0].count()
win_rate = successful_trades / total_trades if total_trades > 0 else 0
sharpe_ratio = df["PnL"].mean() / df["PnL"].std() * np.sqrt(252) if df["PnL"].std() != 0 else 0
max_drawdown = (df["Cumulative_PnL"].cummax() - df["Cumulative_PnL"]).max()

print(f"📊 Общо сделки: {int(total_trades)}")
print(f"✅ Успешни сделки: {successful_trades}")
print(f"📈 Win Rate: {win_rate:.2%}")
print(f"💰 Кумулативна печалба: {df['Cumulative_PnL'].iloc[-1]:.2f}")
print(f"📉 Max Drawdown: {max_drawdown:.2f}")
print(f"📊 Sharpe Ratio: {sharpe_ratio:.2f}")
print("🖼️ Графиката е записана като model_signals_plot.png в Google Drive.")

In [ ]:
for lr in [0.0001, 0.0003, 0.001]:
    for gamma in [0.95, 0.99]:
        model = PPO("MlpPolicy", env, learning_rate=lr, gamma=gamma, verbose=0)
        model.learn(total_timesteps=5000)
        # оцени модела и запиши резултата


In [ ]:
if df.empty:
    print("⚠️ X_test.csv е празен — няма данни за inference.")
else:
    obs = df[features].iloc[[-1]].to_numpy().astype(np.float32)
    model = PPO.load("/content/drive/MyDrive/DTASC/model_retrain_latest.zip")
    action, _ = model.predict(obs, deterministic=True)
    action = int(action[0]) if isinstance(action, np.ndarray) else int(action)
    print("📌 Действие от модела:", "BUY 🟢" if action == 1 else "SELL 🔴")


In [ ]:
import pandas as pd
import numpy as np
from stable_baselines3 import PPO

# 🔁 Зареждане на последния ред от X_test.csv
x_test_path = "/content/drive/MyDrive/DTASC/X_test.csv"
df = pd.read_csv(x_test_path, sep=";")

# 🧠 Очаквани входни характеристики
features = [
    "Open", "High", "Low", "Close", "Volume", "RSI", "MACD",
    "Bollinger_Middle", "Bollinger_Upper", "Bollinger_Lower",
    "Balance", "Price_Change"
]

# ✅ Проверка за липсващи колони
missing = [col for col in features if col not in df.columns]
if missing:
    print(f"⚠️ Липсващи колони в X_test.csv: {missing}")
else:
    obs = df[features].iloc[[-1]].to_numpy().astype(np.float32)

    # 📦 Зареждане на модела
    model = PPO.load("/content/drive/MyDrive/DTASC/model_retrain_latest.zip")

    # 🤖 Предсказване на действие
    action, _ = model.predict(obs, deterministic=True)
    action = int(action[0]) if isinstance(action, np.ndarray) else int(action)

    print("📌 Действие от модела:", "BUY 🟢" if action == 1 else "SELL 🔴")
